# Advent of Code 2021

Never again.

That's what I said in 2019. I both love https://adventofcode.com and hate that the puzzles are so beautifully put together that I can't resist doing one more little one - even though I really don't need anything else going on in my life over the festive season. Perhaps I'll just take a look at day 1... And then...

Well if I'm going to do this I might as well record feelings I've had along the way and any insights into how I could approach other problems better.

I'm no way committing to completing it this year. (Come on - you don't have to do this)

Merry Christmas, J.

In [1]:
from collections import (
    Counter,
    defaultdict,
    deque,
    namedtuple
)

from dataclasses import dataclass
import math
import re

In [2]:
def get_puzzle_input(file, strip_lines=True, remove_blank_lines=True):
    """Return the puzzle input as a list."""
    with open(file) as fh:
        lines = fh.readlines()
    
    if strip_lines:
        lines = [l.strip() for l in lines]
    
    if remove_blank_lines:
        lines = [l for l in lines if l]
        
        
    return lines

## Day 1 Sonar Sweep

https://adventofcode.com/2021/day/1

Count the number of times a depth measurement increases from the previous measurement. (There is no measurement before the first measurement.)

In [4]:
test_input = [
    199,
    200,
    208,
    210,
    200,
    207,
    240,
    269,
    260,
    263
]
real_input = get_puzzle_input('input.txt')
real_input = [int(i) for i in real_input]

In [5]:
def num_increases(puzzle_input):
    num_increases = 0
    current_num = None
    for val in puzzle_input:
        if current_num is not None and val > current_num:
            num_increases += 1
        current_num = val

    return num_increases

In [6]:
num_increases(test_input), num_increases(real_input)

(7, 1754)

### Day 1 Part 2
Instead, consider sums of a three-measurement sliding window

In [3]:
def windows(puzzle_input, window_size):
    output = []
    puzzle_len = len(puzzle_input)
    for puzzle_idx in range(puzzle_len):
        window = []
        for window_idx in range(window_size):
            idx = puzzle_idx + window_idx
            if idx < puzzle_len:
                window.append(puzzle_input[idx])
        output.append(tuple(window))
    return output

In [8]:
windows(test_input, 3)

[(199, 200, 208),
 (200, 208, 210),
 (208, 210, 200),
 (210, 200, 207),
 (200, 207, 240),
 (207, 240, 269),
 (240, 269, 260),
 (269, 260, 263),
 (260, 263),
 (263,)]

In [9]:
windowed_test_input = [sum(x) for x in windows(test_input, 3)]
num_increases(windowed_test_input)

5

In [10]:
windowed_real_input = [sum(x) for x in windows(real_input, 3)]
num_increases(windowed_real_input)

1789

### Day 1 Observations

I skipped last year's AOC and I realise that I always go into each of these puzzles with the expectation that I won't have a clue where to start and everyone else will be able to see the solution. Also interesting that I started on the windowing solution just as my train was pulling into the station with a 'write it and see what happens' mind set; it was turning into a real mess. During the next 10 minute bus ride I had a much clearer idea of what I was going to do. Sometimes it pays to walk away from the keyboard.

## Day 2 Dive!

https://adventofcode.com/2021/day/2

Calculate the horizontal position and depth you would have after following the planned course.

In [11]:
@dataclass(frozen=True)
class Position:
    horizontal: int = 0
    depth: int = 0
    
    def move(self, command):
        movement_map = {
            'forward': (1, 0),
            'down': (0, 1),
            'up': (0, -1),
        }
        input_re = re.compile(r'(\S+) ([0-9-]+)')
        match = input_re.match(command)
        if not match:
            print(command)
        
        direction = match.groups()[0]
        amount = int(match.groups()[1])
        
        diff = movement_map[direction]
        return Position(
            self.horizontal + diff[0] * amount,
            self.depth + diff[1] * amount,
        )


In [14]:
assert Position(0, 0).move('down 12') == Position(0, 12)
assert Position(0, 12).move('up 12') == Position(0, 0)
assert Position(100, 12).move('forward 12') == Position(112, 12)

In [17]:
test_directions = [
    'forward 5',
    'down 5',
    'forward 8',
    'up 3',
    'down 8',
    'forward 2'
]
real_directions = get_puzzle_input('input_2.txt')

In [20]:
def play(directions, initial_position):
    position = initial_position
    for dir in directions:
        position = position.move(dir)
    return position.horizontal * position.depth


In [22]:
play(test_directions, Position(0, 0)), play(real_directions, Position(0, 0))

(150, 1524750)

### Day 2 Part 2

Calculate using the new interpretation as up and down as tilt.

In [24]:
@dataclass(frozen=True)
class Position2:
    horizontal: int = 0
    depth: int = 0
    aim: int = 0
    
    def move(self, command):
        movement_map = {
            'forward': (1, self.aim, 0),
            'down': (0, 0, 1),
            'up': (0, 0, -1),
        }
        input_re = re.compile(r'(\S+) ([0-9-]+)')
        match = input_re.match(command)
        if not match:
            print(command)
        
        direction = match.groups()[0]
        amount = int(match.groups()[1])
        
        diff = movement_map[direction]
        return Position2(
            self.horizontal + diff[0] * amount,
            self.depth + diff[1] * amount,
            self.aim + diff[2] * amount,
        )

In [25]:
play(test_directions, Position2(0, 0)), play(real_directions, Position2(0, 0))

(900, 1592426537)

### Day 2 Observations

Frozen dataclasses have been the killer missing feature in Python. Once you've worked with Scala case classes or Kotlin data classes, you can't imagine why a language wouldn't have such a feature (I guess I wasn't aware of them when doing 2018 AOC!)

## Day 3 Binary Diagnostic

https://adventofcode.com/2021/day/3

Use the binary numbers in your diagnostic report to calculate the gamma rate and epsilon rate, then multiply them together. What is the power consumption of the submarine? (Be sure to represent your answer in decimal, not binary.)

In [26]:
test_input = [
    '00100',
    '11110',
    '10110',
    '10111',
    '10101',
    '01111',
    '00111',
    '11100',
    '10000',
    '11001',
    '00010',
    '01010',
]
real_input = get_puzzle_input('input3.txt')

In [27]:
def gamma_rate(input):
    num_numbers = len(input)
    num_figures = len(input[0])
    running_count = [0] * num_figures
    binaries = [int(x, base=2) for x in input]
    
    for binary in binaries:
        for figure in range(num_figures):
            mask = 2**figure
            is_set = (binary & mask) // mask
            running_count[figure] += is_set

    return sum(
        val * 2 // num_numbers * 2**idx
        for (idx, val)
        in enumerate(running_count)
    )

In [28]:
def epsilon_rate(gamma_rate, num_figures):
    return (2**num_figures - 1) ^ gamma_rate

In [31]:
test_gamma_rate = gamma_rate(test_input)
test_epsilon_rate = epsilon_rate(test_gamma_rate, len(test_input[0]))
test_gamma_rate, test_epsilon_rate, test_gamma_rate * test_epsilon_rate


(22, 9, 198)

In [32]:
real_gamma_rate = gamma_rate(real_input)
real_epsilon_rate = epsilon_rate(real_gamma_rate, len(real_input[0]))
real_gamma_rate, real_epsilon_rate, real_gamma_rate * real_epsilon_rate

(3808, 287, 1092896)

### Day 3, part 2

Use the binary numbers in your diagnostic report to calculate the oxygen generator rating and CO2 scrubber rating, then multiply them together. What is the life support rating of the submarine? (Be sure to represent your answer in decimal, not binary.)

In [33]:
def figure_filter(figures, figure_idx, is_most_common): 
    running_count = 0
    num_numbers = len(figures)
    mask = 2**figure_idx
    ones = []
    zeroes = []

    for figure in figures:
        is_set = (figure & mask) // mask
        running_count += is_set
        if is_set:
            ones.append(figure)
        else:
            zeroes.append(figure)

    more_ones = running_count * 2 >= num_numbers
    
    if (
        more_ones and is_most_common or
        not more_ones and not is_most_common
    ):
        return ones
    else:
        return zeroes

In [34]:
def part2(figures, is_most_common):
    int_figures = [int(x, base=2) for x in figures]
    num_figures = len(figures[0])
    # Count down to look at the most sig bit first
    for figure_idx in range(num_figures - 1, -1, -1):
        int_figures = figure_filter(int_figures, figure_idx, is_most_common)
        
        if len(int_figures) <= 1:
            break
    
    return int_figures 

In [36]:
oxygen_rating = part2(test_input, True)
co_rating = part2(test_input, False)
oxygen_rating, co_rating, oxygen_rating[0] * co_rating[0]

([23], [10], 230)

In [38]:
oxygen_rating = part2(real_input, True)
co_rating = part2(real_input, False)
oxygen_rating, co_rating, oxygen_rating[0] * co_rating[0]

([3443], [1357], 4672151)

### Day 3 Observations

Think I made the right call in working with ints rather than strings in part 1.


## Day 4 Giant Squid

https://adventofcode.com/2021/day/4

To guarantee victory against the giant squid, figure out which board will win first. What will your final score be if you choose that board?

In [39]:
test_input = get_puzzle_input('test_input_4.txt', remove_blank_lines=False)
real_input = get_puzzle_input('input_4.txt', remove_blank_lines=False)

In [40]:
class Board:
    def __init__(self, grid):
        self.grid_size = len(grid)
        self.parse_grid(grid)
        self.reset()

    def parse_grid(self, grid):
        self.numbers = {}
        self.locations = {}
        for row_idx, line in enumerate(grid):
            row = re.split('\s+', line)
            for col_idx, num in enumerate(row):
                location = (col_idx, row_idx)
                num = int(num)
                self.numbers[num] = location
                self.locations[location] = num
    
    def reset(self):
        self.dabbed = Counter()
        self.unmarked = set(self.numbers.keys())
        
    def dab(self, number):
        if number not in self.unmarked:
            return None
        
        self.unmarked.remove(number)

        location = self.numbers[number]
        
        col_idx = ('COL', location[0])
        row_idx = ('ROW', location[1])
        self.dabbed[col_idx] += 1
        self.dabbed[row_idx] += 1
        
        col_win = self.dabbed[col_idx] == self.grid_size
        row_win = self.dabbed[row_idx] == self.grid_size

        if not col_win and not row_win:
            return None

        winning_line = []
        
        for line_idx in range(self.grid_size):
            num_location = (location[0], line_idx) if col_win else (line_idx, location[1])
            winning_line.append(self.locations[num_location])
        
        return winning_line     

In [41]:
# Test winning row
input = [
    '14 21 17 24  4',
    '10 16 15  9 19',
    '18  8 23 26 20',
    '22 11 13  6  5',
    '2  0 12  3  7'
]
board = Board(input)
for num in [14, 21, 17, 24]:
    board.dab(num)
assert board.dab(4) == [14, 21, 17, 24, 4]

In [42]:
# Test winning col
board.reset()
for num in [17, 15, 13, 12]:
    board.dab(num)
assert board.dab(23) == [17, 15, 23, 13, 12]

In [43]:
class Game:
    def __init__(self, lines):
        self.parse_input(lines)

    def parse_input(self, lines):
        self.numbers = [int(i) for i in lines[0].split(',')]
        self.boards = []
        grids = deque(lines[1:])
        while len(grids) >= 6:
            grids.popleft()
            grid = [grids.popleft() for _ in range(5)]
            self.boards.append(Board(grid))
        
        self.in_play_boards = set(range(len(self.boards)))
    
    def play(self):
        for num in self.numbers:
            for board in self.boards:
                win = board.dab(num)
                if win:
                    return num, win, sum(board.unmarked)

    def play_part_2(self):
        for num in self.numbers:
            for board_idx in sorted(list(self.in_play_boards)):
                board = self.boards[board_idx]
                win = board.dab(num)
                if win:
                    # print(f'{num} {board_idx} {win} {sum(board.unmarked)}')
                    self.in_play_boards.remove(board_idx)
                    if not self.in_play_boards:
                        return num, win, sum(board.unmarked)

In [46]:
test_game = Game(test_input)
test_game.play()

(24, [14, 21, 17, 24, 4], 188)

In [47]:
real_game = Game(real_input)
real_game.play()

(35, [75, 95, 35, 6, 47], 726)

In [48]:
35 * 726

25410

### Day 4, part 2

Figure out which board will win last. Once it wins, what would its final score be?

In [49]:
test_game = Game(test_input)
test_game.play_part_2()

(13, [0, 13, 7, 10, 16], 148)

In [50]:
real_game = Game(real_input)
real_game.play_part_2()

(14, [95, 18, 52, 14, 22], 195)

In [51]:
14 * 195

2730

### Day 4 Observations

Make sure test and production run the same code. I originally was loading the test input from the command line and the real input from a file. I had forgotten that this was stripping blank lines by default hence the Bingo boards were being mixed up. This was doubly confusing given that the first part gave the correct answer by chance and the bug was only found in running the second part which assumed we were developing on solid ground.

## Day 5 Hydrothermal Venture

https://adventofcode.com/2021/day/5

To avoid the most dangerous areas, you need to determine the number of points where at least two lines overlap. In the above example, this is anywhere in the diagram with a 2 or larger - a total of 5 points.

Consider only horizontal and vertical lines. At how many points do at least two lines overlap?

In [52]:
test_input = get_puzzle_input('test_input_5.txt')
real_input = get_puzzle_input('input_5.txt')

In [55]:
Point = namedtuple('Point', ('x', 'y'))

In [56]:
# Part 1
def intersected_points(point_a, point_b):
    lower, upper = sorted([point_a, point_b])
    if upper.x > lower.x:
        return [Point(x, upper.y) for x in range(lower.x, upper.x + 1)]
    
    return [Point(upper.x, y) for y in range(lower.y, upper.y + 1)]

In [57]:
# Part 2 - handles diagonal lines
def intersected_points(point_a, point_b):
    line_len = 1
    if point_a.x > point_b.x:
        x_incr = -1
        line_len = point_a.x - point_b.x
    elif point_a.x < point_b.x:
        x_incr = 1
        line_len = point_b.x - point_a.x
    else:
        x_incr = 0

    if point_a.y > point_b.y:
        y_incr = -1
        line_len = point_a.y - point_b.y
    elif point_a.y < point_b.y:
        y_incr = 1
        line_len = point_b.y - point_a.y
    else:
        y_incr = 0

    return {
        Point(point_a.x + x_incr * val, point_a.y + y_incr * val)
        for val
        in range(line_len + 1)
    } 

In [58]:
assert intersected_points(Point(1, 2), Point(1, 4)) == {Point(x=1, y=2), Point(x=1, y=3), Point(x=1, y=4)}
assert intersected_points(Point(1, 4), Point(1, 2)) == {Point(x=1, y=2), Point(x=1, y=3), Point(x=1, y=4)}
assert intersected_points(Point(1, 4), Point(1, 4)) == {Point(x=1, y=4)}
assert intersected_points(Point(10, 4), Point(12, 4)) == {Point(x=10, y=4), Point(x=11, y=4), Point(x=12, y=4)}
assert intersected_points(Point(1, 2), Point(4, 5)) == {Point(x=1, y=2), Point(x=2, y=3), Point(x=3, y=4), Point(x=4, y=5)}

In [59]:
class Game:
    regexp = re.compile('^([-0-9]+),([-0-9]+) -> ([-0-9]+),([-0-9]+)$')

    def __init__(self, input):
        self.parse_input(input)
    
    def parse_input(self, puzzle_input):
        self.lines = []
        for line in puzzle_input:
            match = self.regexp.match(line)
            if match:
                groups = match.groups()
                point_a = Point(int(groups[0]), int(groups[1]))
                point_b = Point(int(groups[2]), int(groups[3]))
                self.lines.append((point_a, point_b))

    def play(self, ignore_diagonals=True):
        points = Counter()
        for (point_a, point_b) in self.lines:
            if ignore_diagonals:
                # Part 1
                if point_a.x != point_b.x and point_a.y != point_b.y:
                    continue
            for point in intersected_points(point_a, point_b):
                points[point] += 1
        
        danger_points = [
            point
            for (point, val)
            in points.items()
            if val > 1
        ]
        
        return len(danger_points)

In [60]:
test_game = Game(test_input)
test_game.play()

5

In [61]:
real_game = Game(real_input)
real_game.play()

5092

### Day 5 Part 2

You still need to determine the number of points where at least two lines overlap. In the above example, this is still anywhere in the diagram with a 2 or larger - now a total of 12 points.

Consider all of the lines. At how many points do at least two lines overlap?

In [62]:
test_game = Game(test_input)
test_game.play(ignore_diagonals=False)

12

In [63]:
real_game = Game(real_input)
real_game.play(ignore_diagonals=False)

20484

### Day 5 Observations

Intersecting points implementation is a bit clunky but - hey ho.

## Day 6 Lantern Fish

https://adventofcode.com/2021/day/6

In this example, after 18 days, there are a total of 26 fish. After 80 days, there would be a total of 5934.

Find a way to simulate lanternfish. How many lanternfish would there be after 80 days?

In [67]:
class LanternFish:
    CYCLE_LENGTH = 6
    DEFAULT_START = 8

    def __init__(self, start_of_cycle=DEFAULT_START):
        self.start_of_cycle = start_of_cycle
        self.current = start_of_cycle

    def new_day(self):
        self.current -= 1
        if self.current == -1:
            self.current = self.CYCLE_LENGTH
            child = LanternFish()
            return child
        else:
            return None

In [68]:
class SingleFishGame:
    """Only need to solve for each unique initial state as any similar fish
    will result in the same number of descendents.
    """
    def __init__(self, start_of_cycle):
        self.fishes = [LanternFish(start_of_cycle=start_of_cycle)]
    
    def play(self, num_days):
        for _ in range(num_days):
            new_fish = []
            for fish in self.fishes:
                child = fish.new_day()
                if child:
                    new_fish.append(child)
            
            self.fishes.extend(new_fish)
        
        return len(self.fishes) 

In [69]:
%%time

num_descendents_per_state = {}

# Loop over all the possible initial states
for i in range(7):
    game = SingleFishGame(i)
    total = game.play(80)
    num_descendents_per_state[i] = total

num_descendents_per_state

CPU times: user 35.8 ms, sys: 1.69 ms, total: 37.5 ms
Wall time: 36 ms


{0: 1421, 1: 1401, 2: 1191, 3: 1154, 4: 1034, 5: 950, 6: 905}

In [70]:
initial_count = Counter()
for i in [5,3,2,2,1,1,4,1,5,5,1,3,1,5,1,2,1,4,1,2,1,2,1,4,2,4,1,5,1,3,5,4,3,3,1,4,1,3,4,4,1,5,4,3,3,2,5,1,1,3,1,4,3,2,2,3,1,3,1,3,1,5,3,5,1,3,1,4,2,1,4,1,5,5,5,2,4,2,1,4,1,3,5,5,1,4,1,1,4,2,2,1,3,1,1,1,1,3,4,1,4,1,1,1,4,4,4,1,3,1,3,4,1,4,1,2,2,2,5,4,1,3,1,2,1,4,1,4,5,2,4,5,4,1,2,1,4,2,2,2,1,3,5,2,5,1,1,4,5,4,3,2,4,1,5,2,2,5,1,4,1,5,1,3,5,1,2,1,1,1,5,4,4,5,1,1,1,4,1,3,3,5,5,1,5,2,1,1,3,1,1,3,2,3,4,4,1,5,5,3,2,1,1,1,4,3,1,3,3,1,1,2,2,1,2,2,2,1,1,5,1,2,2,5,2,4,1,1,2,4,1,2,3,4,1,2,1,2,4,2,1,1,5,3,1,4,4,4,1,5,2,3,4,4,1,5,1,2,2,4,1,1,2,1,1,1,1,5,1,3,3,1,1,1,1,4,1,2,2,5,1,2,1,3,4,1,3,4,3,3,1,1,5,5,5,2,4,3,1,4]:
    initial_count[i] += 1
    
initial_count

Counter({5: 42, 3: 43, 2: 52, 1: 109, 4: 54})

In [71]:
sum(count * num_descendents_per_state[state] for state, count in initial_count.items())

359999

### Day 6 Part 2

And on day 256 how many will there be?

In [72]:
class LanternFish:
    """Any fish born on the same day will have the same number of descendents
    so only need to track one."""
    CYCLE_LENGTH = 6
    DEFAULT_START = 8

    def __init__(self, start_of_cycle=DEFAULT_START):
        self.start_of_cycle = start_of_cycle
        self.current = start_of_cycle
        self.untracked_parents = []
        self.num_descendents = 0

    def gives_birth(self):
        self.current -= 1
        if self.current == -1:
            self.current = self.CYCLE_LENGTH
            return True
        else:
            return False
    
    def add_parent(self, parent):
        self.untracked_parents.append(parent)
        parent.add_descendent()

    def add_descendent(self):
        self.num_descendents += 1
        for parent in self.untracked_parents:
            parent.add_descendent()

In [73]:
class SingleFishGame:
    def __init__(self, start_of_cycle):
        self.fishes = [LanternFish(start_of_cycle=start_of_cycle)]
        self.day = 0
    
    def play(self, num_days):
        for day_num in range(num_days):
            new_fish = []
            child = None
            for fish in self.fishes:
                gives_birth = fish.gives_birth()
                if not gives_birth:
                    continue
                
                if child:
                    child.add_parent(fish)
                else:
                    child = LanternFish()
                    new_fish.append(child)
            
            self.fishes.extend(new_fish)
        
        return len(self.fishes) + sum(fish.num_descendents for fish in game.fishes)

In [75]:
%%time

part_2_num_descendents_per_state = {}
for i in range(1, 6):
    print(f'Calculating for initial state {i}...')
    game = SingleFishGame(i)
    total = game.play(256)
    part_2_num_descendents_per_state[i] = total

part_2_num_descendents_per_state

Calculating for initial state 1...
Calculating for initial state 2...
Calculating for initial state 3...
Calculating for initial state 4...
Calculating for initial state 5...
CPU times: user 1h 6min 9s, sys: 3.29 s, total: 1h 6min 13s
Wall time: 1h 6min 16s


{1: 6206821033, 2: 5617089148, 3: 5217223242, 4: 4726100874, 5: 4368232009}

In [77]:
sum(count * part_2_num_descendents_per_state[state] for state, count in initial_count.items())

1631647919273

### Day 6 Observations

Returns in one hour - wow. As I was running it for the first index I could see that it was going to return (from printing out the day being calculated). I don't think there was a need to rerun it for all the initial states - Could have, for instance derived state 2 from state 1 by eliminating all state 8s (newborns) and likewise for state 3 - not counting 7s and 8s. Perhaps I could say that state 2 on day 265 is state 1 on day 264 etc so only need one run through. Could have also looked for repeated patterns - but - there comes the point when you're bunged up with a cold and you're happy to have breakfast and wait.

### Day 6 Revisited

A few days later I had a look at this again and realised a far easier approach of mapping states to future states a week down the line. So in general given a fish in state n, in a weeks time it will result in a fish in state n (itself) and a fish in state n+2 (its child) unless the initial state is 7 or 8 (a newborn)

In [2]:
def num_fish_from_state(initial_state, num_days):
    seven_day_state_map = {
        0: (0, 2),
        1: (1, 3),
        2: (2, 4),
        3: (3, 5),
        4: (4, 6),
        5: (5, 7),
        6: (6, 8),
        7: (0, ),
        8: (1, ),
    }
    counts = [0] * 9
    
    num_weeks = num_days // 7
    remaining_days = num_days - num_weeks * 7
    
    initial_state -= remaining_days
    counts[initial_state % 7] = 1
    if initial_state < 0:
        # The newly spawned fish
        counts[8 + initial_state + 1] = 1
    
    # Go up in weeks
    for i in range(num_weeks):
        for state, num in enumerate(list(counts)):
            new_states = seven_day_state_map[state]
            counts[state] -= num
            for new_state in new_states:
                counts[new_state] += num
    
    return sum(counts)
  

In [3]:
%%time

part_2_lookup_improved = {}

for i in range(1, 6):
    part_2_lookup_improved[i] = num_fish_from_state(i, 256)

part_2_lookup_improved

CPU times: user 1.7 ms, sys: 7 µs, total: 1.71 ms
Wall time: 1.71 ms


{1: 6206821033, 2: 5617089148, 3: 5217223242, 4: 4726100874, 5: 4368232009}

From 1 hour to 1 millisecond! It's interesting that the puzzle setter went for 256 days. If I think about the question without thinking of the implementation I might have been tempted on a year (365 days). If it were a year the solution I went for on the day wouldn't have returned. On the otherhand if it were any less I wouldn't have been suitably punished for a slow implementation. If it were 200 days for example - my solution would have returned in under 30 seconds and returned in 6 minutes for 230 days - both being far quicker to wait than to rewrite. 256 seems a sweet spot that allows less lazy people the oportunity to rewrite and come up with a more elegant and satisfying solution whilst keeping us lazy folk in the game.

## Day 7 The Treachery of Whales

https://adventofcode.com/2021/day/7

Determine the horizontal position that the crabs can align to using the least fuel possible. How much fuel must they spend to align to that position?

In [78]:
test_positions = [16,1,2,0,4,2,7,1,2,14]
real_positions = [1101,1,29,67,1102,0,1,65,1008,65,35,66,1005,66,28,1,67,65,20,4,0,1001,65,1,65,1106,0,8,99,35,67,101,99,105,32,110,39,101,115,116,32,112,97,115,32,117,110,101,32,105,110,116,99,111,100,101,32,112,114,111,103,114,97,109,10,485,546,350,100,791,199,115,144,649,41,1656,163,903,71,384,30,2,251,554,210,434,206,546,759,258,54,1478,48,438,601,326,5,1017,165,168,201,622,864,1338,24,1074,545,499,484,264,345,332,869,297,711,674,346,1139,317,875,242,725,250,1619,1408,956,380,366,187,1034,1555,467,170,114,1136,150,183,304,44,37,333,791,34,540,716,1923,342,6,922,18,24,1189,59,1726,636,442,426,1089,526,298,386,296,623,80,272,240,406,628,238,409,302,35,404,92,48,157,1545,409,1382,151,1656,3,76,14,115,566,650,197,448,573,161,86,140,875,128,319,4,822,530,189,247,667,82,316,274,110,206,1012,166,639,579,459,284,200,16,24,147,743,113,1562,387,60,84,797,14,30,1015,508,88,113,685,658,257,1507,348,30,808,416,9,835,671,16,474,885,230,47,463,1324,1263,183,603,739,0,296,789,1411,339,27,1154,31,882,409,646,92,153,147,974,497,308,85,311,135,627,811,295,698,2,20,1170,789,702,1194,1390,432,257,715,958,150,1295,144,1193,607,67,929,383,1051,1231,393,190,380,1203,1090,1238,143,206,210,1004,304,1305,392,143,1379,665,806,452,185,4,1,201,1104,633,274,493,472,141,674,1261,106,587,244,903,91,158,69,137,922,778,143,692,160,474,7,304,824,657,15,1110,806,295,1565,1162,358,725,877,440,690,13,69,111,304,300,493,249,105,746,20,163,561,913,558,252,13,193,508,12,845,120,205,154,1582,349,1471,529,268,23,689,6,776,565,401,0,623,186,62,95,148,275,1,137,320,0,19,1803,10,100,652,750,226,484,180,46,310,446,667,543,277,139,265,74,171,87,1753,337,162,59,1339,1040,1287,1084,192,169,50,1557,81,1120,271,167,977,76,295,12,54,710,36,364,521,989,1634,720,1031,1204,355,380,859,633,223,1207,221,31,138,1305,779,1026,52,92,216,221,0,980,130,1197,585,1213,63,157,213,993,1123,588,450,256,1021,90,1420,47,386,843,1188,1466,807,596,416,23,32,62,1289,317,368,491,907,1386,114,1620,39,344,1342,43,281,12,1202,257,1357,203,465,174,350,833,125,54,390,687,339,628,819,261,1341,840,643,414,82,373,428,1315,570,1070,686,893,70,728,70,358,1233,189,1247,244,1043,1135,42,531,962,35,30,1462,946,856,145,386,1134,1071,379,740,175,1205,234,354,5,1028,506,58,433,1055,749,854,99,298,1248,619,62,181,258,42,130,1698,1313,672,129,222,127,636,846,24,1324,946,622,689,168,329,301,458,173,591,772,93,282,8,320,106,233,412,556,2,522,369,8,1371,899,503,568,667,1199,92,115,899,952,81,629,175,274,763,204,339,236,317,257,731,1082,1724,211,516,165,91,334,1216,101,21,1340,235,336,1351,723,1745,183,841,104,172,1080,180,493,798,1468,45,1627,59,58,368,560,166,1125,136,26,1238,1580,420,1732,155,55,293,751,194,1723,175,11,30,10,307,57,66,704,285,685,241,565,368,50,181,1047,147,420,1341,20,37,400,798,476,1060,642,134,140,502,254,997,910,636,179,22,612,55,237,258,48,205,412,155,910,192,262,9,91,766,1426,71,5,315,285,186,629,422,1289,397,52,860,1390,106,887,1285,1196,684,36,703,199,4,277,151,82,293,1047,455,21,935,630,736,118,13,30,584,453,1446,381,585,810,177,1028,280,281,184,78,673,126,410,872,524,78,188,121,394,201,1764,609,350,706,428,88,783,189,643,305,516,259,582,309,985,338,21,235,73,44,585,71,983,175,1336,1056,10,8,537,701,1653,657,70,1242,442,52,973,203,173,959,964,272,348,3,567,714,1466,382,129,613,1042,686,461,57,523,740,726,149,1490,867,44,379,1270,547,649,1103,912,1354,985,458,887,603,1016,317,499,690,829,1231,364,772,29,57,357,467,484,202,150,109,95,414,444,383,62,124,645,723,772,881,1553,413,123,248,1085,453,260,214,113,1874,482,942,235,899,122,171,127,913,424,406,49,97,1848,295,1152,111,350,54,1160,2,16,156,448,394,740,49,1237,548,206,1206,775,748,728,48,238,148,109,18,56,64,515,163,609,273,301,396,207,51,478,1183,864,772,450,222,1387,269,40,87,426,164,1270,21,347,316,331,408,914,1046,173,48,398,177,431,47,1055,221,513,226,84,285,566,270,333,343,480,1802,101,683,168,1347,582,80,22,329,350,108,379,14,53,349,43,435,195,102,168,338]

In [79]:
len(real_positions), max(real_positions), min(real_positions)

(1000, 1923, 0)

OK, only roughly 2 million calculations to brute force every position. Let's start there

In [80]:
def crab_fuel_used_part_1(curr_pos, new_pos):
    return abs(curr_pos - new_pos)

def crab_fuel_used_part_2(curr_pos, new_pos):
    diff = abs(curr_pos - new_pos)
    return diff * (diff + 1) // 2

def best_alignment(positions, crab_fuel_used):
    best_pos = None
    best_fuel_used = None
    max_pos = max(positions)
    min_pos = min(positions)
    
    for x in range (min_pos, max_pos + 1):
        fuel_used = sum(crab_fuel_used(pos, x) for pos in positions)
        if best_fuel_used is None or fuel_used < best_fuel_used:
            best_pos = x
            best_fuel_used = fuel_used
    
    return best_pos, best_fuel_used

In [81]:
best_alignment(test_positions, crab_fuel_used_part_1), best_alignment(real_positions, crab_fuel_used_part_1)

((2, 37), (339, 343468))

### Day 7 Part 2

And now the crabs use an extra unit of fuel every time they move

In [82]:
best_alignment(test_positions, crab_fuel_used_part_2), best_alignment(real_positions, crab_fuel_used_part_2)

((5, 168), (478, 96086265))

### Day 7 Observations

Suspiciously straight forward - the calm before the storm? We are heading towards a maze! I thought the strategy pattern worked quite well here.

## Day 8 Seven Segment Search

https://adventofcode.com/2021/day/8

Because the digits 1, 4, 7, and 8 each use a unique number of segments, you should be able to tell which combinations of signals correspond to those digits. Counting only digits in the output values (the part after | on each line), in the above example, there are 26 instances of digits that use a unique number of segments (highlighted above).

In the output values, how many times do digits 1, 4, 7, or 8 appear?

In [1]:
def get_output_digits(puzzle_line):
    output = puzzle_line.split(' ')[11:]
    return [frozenset(s) for s in output]

def get_input_signals(puzzle_line):
    input = puzzle_line.split(' ')[:10]
    return [frozenset(s) for s in input]  

In [4]:
test_input = get_puzzle_input('test_input_8.txt')
real_input = get_puzzle_input('input_8.txt')

In [5]:
def solve_part_1(puzzle_input):
    unique_num_count = 0
    for line in puzzle_input:
        output_digits = get_output_digits(line)
        for digit in output_digits:
            if len(digit) in (2, 3, 4, 7):
                unique_num_count += 1
    
    return unique_num_count

In [6]:
solve_part_1(test_input), solve_part_1(real_input)

(26, 264)

### Day 8, part 2

```
Considering the line
acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab | cdfeb fcadb cdfeb cdbaf

After some careful analysis, the mapping between signal wires and segments only make sense in the following configuration:

 dddd
e    a
e    a
 ffff
g    b
g    b
 cccc

So, the unique signal patterns would correspond to the following digits:

acedgfb: 8
cdfbe: 5
gcdfa: 2
fbcad: 3
dab: 7
cefabd: 9
cdfgeb: 6
eafb: 4
cagedb: 0
ab: 1
```

For each entry, determine all of the wire/segment connections and decode the four-digit output values. What do you get if you add up all of the output values?

In [8]:
def get_digit_map(signal_pattern):
    one = [s for s in signal_pattern if len(s) == 2][0]
    four = [s for s in signal_pattern if len(s) == 4][0]
    seven = [s for s in signal_pattern if len(s) == 3][0]
    eight = [s for s in signal_pattern if len(s) == 7][0]
    
    # 0, 6 and 9
    six_segments = [s for s in signal_pattern if len(s) == 6]
    
    six = [s for s in six_segments if s & seven != seven][0]
    six_segments.remove(six)
    nine = [s for s in six_segments if s & four == four][0]
    six_segments.remove(nine)
    zero = six_segments[0]

    # 3, 2 and 5
    five_segments = [s for s in signal_pattern if len(s) == 5]
    five = [s for s in five_segments if len(six - s)  == 1][0]
    five_segments.remove(five)
    
    three = [s for s in five_segments if s & one == one][0]
    five_segments.remove(three)
    two = five_segments[0]
    
    return {
        zero: 0,
        one: 1,
        two: 2,
        three: 3,
        four: 4,
        five: 5,
        six: 6,
        seven: 7,
        eight: 8,
        nine: 9
    }

In [9]:
# From the above example:
test_line = 'acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab | cdfeb fcadb cdfeb cdbaf'
input_signals = get_input_signals(test_line)
digit_map = get_digit_map(input_signals)

expected = {
    frozenset('acedgfb'): 8,
    frozenset('cdfbe'): 5,
    frozenset('gcdfa'): 2,
    frozenset('fbcad'): 3,
    frozenset('dab'): 7,
    frozenset('cefabd'): 9,
    frozenset('cdfgeb'): 6,
    frozenset('eafb'): 4,
    frozenset('cagedb'): 0,
    frozenset('ab'): 1,
}

assert sorted(digit_map.items(), key=lambda x: x[1]) == sorted(expected.items(), key=lambda x: x[1])

In [10]:
def solve_part_2(puzzle_input):
    total = 0
    for line in puzzle_input:
        input_signals = get_input_signals(line)
        output_digits = get_output_digits(line)
        
        digit_map = get_digit_map(input_signals)
        
        decoded_digits = [digit_map[d] for d in output_digits]
        decoded_figure = int(''.join(map(str, decoded_digits)))
        total += decoded_figure
    
    return total

In [11]:
solve_part_2(test_input), solve_part_2(real_input)

(61229, 1063760)

### Day 8 Observations

We have a tech talk session on Wednesdays - so I thought for the Christmas special that we'd do today's AOC puzzle as a group swarm programming session. The session's only half an hour so I wanted to check what was achievable to explain and complete in that time before hand (that is to say I cheated and completed it prior to the session.) To fit in the time and to choose something interesting I concentrated on part 2 with only the `get_digit_map` function (the hard bit) not implemented. So, there was a lot of boilerplate already there to test and execute the function. Here's some observations from the session:

- Some people just really don't like puzzles. Problem solving as a group can ellicit a range of negative emotions - from a more neutral - just a waste of time, to feeling the pressure of performing in front of peers. It's important to give people an out by either saying what the session is going to be about up front (I didn't) or telling people that it's fine to drop off if they want to get the time back (I did.)

- The read though and part one of these puzzles are both really important. You think through possible implementations as you're reading it; you're potentially second guessing what's going to be in the second part. Completing the first part gives you a good feel for what you're working with, and you take all this into the second part. Tackling one aspect of the second part involved me paraphrasing the problem up to that point to the group and together with the boilerplate code they had to jump right in at the deep end without a chance to really think it through. This gap in prior knowledge also either made it look like I was exceptionally clever (unlikely) or that I was pretending to be exceptionally clever (very likely). If I were doing it again I think I would do it blind on part 1 of a problem; giving everyone five minutes to read it through first. I'd obviously need to put to one side my own fear of failing in front of peers - failing to solve it can be just as interesting!

- Even though they were thrown in at the deep end - they did it! (their solution is above). I really tried not to prompt - honest - just to type.

## Day 9 Smoke Basin

https://adventofcode.com/2021/day/9

The risk level of a low point is 1 plus its height. In the above example, the risk levels of the low points are 2, 1, 6, and 6. The sum of the risk levels of all low points in the heightmap is therefore 15.

Find all of the low points on your heightmap. What is the sum of the risk levels of all low points on your heightmap?

In [8]:
test_input = get_puzzle_input('test_input_9.txt')
real_input = get_puzzle_input('input_9.txt')

In [9]:
@dataclass(frozen=True)
class Point:
    x: int
    y: int
    
    @property
    def adjacent_points(self):
        return [
            Point(self.x - 1, self.y),
            Point(self.x + 1, self.y),
            Point(self.x, self.y + 1),
            Point(self.x, self.y - 1),
        ]


In [10]:
class Grid:
    def __init__(self, puzzle_input):
        self.points = {}
        self.max_x = len(puzzle_input[0]) - 1
        self.max_y = len(puzzle_input) - 1

        for y, line in enumerate(puzzle_input):
            for x, val in enumerate(line):
                self.points[Point(x, y)] = int(val)
    
    def get_adjacent_points(self, point):
            adjacent_points = point.adjacent_points
            adjacent_points = [
                point
                for point in adjacent_points
                if point.x >= 0 and point.x <= self.max_x
            ]
            adjacent_points = [
                point
                for point in adjacent_points
                if point.y >= 0 and point.y <= self.max_y
            ]
            return adjacent_points
        

    def play_part_1(self):
        risk = 0
        for point, val in self.points.items():
            adjacent_values = [
                self.points[point]
                for point
                in self.get_adjacent_points(point)
            ]
            
            if all([val < adjacent_val for adjacent_val in adjacent_values]):
                risk += 1 + val
        
        return risk

    def find_basin(self, remaining_points):
        # pick a starting point
        basin = {list(remaining_points)[0]}
        
        while True:
            added_points = set()
            for point in basin:
                adjacent_points = self.get_adjacent_points(point)
                adjacent_points = [point for point in adjacent_points if point in remaining_points]
                adjacent_points = {point for point in adjacent_points if self.points[point] < 9}
                added_points = added_points | adjacent_points

            new_basin = basin | added_points
            if new_basin == basin:
                return new_basin

            basin = new_basin
    
    def find_basins(self):
        basins = []
        remaining_points = {point for point, val in self.points.items() if val < 9}
        
        while remaining_points:
            basin = self.find_basin(remaining_points)
            basins.append(basin)
            remaining_points -= basin
        
        return basins 
    
    def play_part_2(self):
        basins = self.find_basins()
        sizes = [len(basin) for basin in basins]
        sizes = list(reversed(sorted(sizes)))
        return sizes[0] * sizes[1] * sizes[2]

In [11]:
test_grid = Grid(test_input)
test_grid.play_part_1()

15

In [12]:
real_grid = Grid(real_input)
real_grid.play_part_1()

478

### Day 9 part 2

Find the three largest basins and multiply their sizes together. In the above example, this is 9 * 14 * 9 = 1134.

What do you get if you multiply together the sizes of the three largest basins?

In [13]:
test_grid = Grid(test_input)
test_grid.play_part_2()

1134

In [14]:
real_grid = Grid(real_input)
real_grid.play_part_2()

1327014

### Day 9 Observations

I find it difficult to spell the word, 'adjacent'.

## Day 10 Syntax Scoring

https://adventofcode.com/2021/day/10

In the above example, an illegal ) was found twice (2*3 = 6 points), an illegal ] was found once (57 points), an illegal } was found once (1197 points), and an illegal > was found once (25137 points). So, the total syntax error score for this file is 6+57+1197+25137 = 26397 points!

Find the first illegal character in each corrupted line of the navigation subsystem. What is the total syntax error score for those errors?

In [4]:
test_input = get_puzzle_input('test_input_10.txt')
real_input = get_puzzle_input('input_10.txt')

In [5]:
def find_syntax_error(line):
    stack = []
    pair_map = {
        '{': '}',
        '(': ')',
        '[': ']',
        '<': '>', 
    }
    close_map = {v: n for (n, v) in pair_map.items()}
    openings = set(pair_map.keys())
    endings = set(pair_map.values())
    
    for char in line:
        if char in openings:
            stack.append(char)
        
        if char in endings:
            if not stack:
                return (None, char)
            opening = stack.pop()
            expected = pair_map[opening]
            if char != expected:
                return (expected, char)
        
    return None

In [6]:
assert find_syntax_error('{}') is None
assert find_syntax_error('{]') == ('}', ']')

In [7]:
def play_part_1(input):
    total = 0
    scores = {
        ')': 3,
        ']': 57,
        '}': 1197,
        '>': 25137
    }
    for line in input:
        error = find_syntax_error(line)
        if error:
            error_char = error[1]
            score = scores[error_char]
            total += score
    
    return total

In [8]:
play_part_1(test_input), play_part_1(real_input)

(26397, 288291)

### Day 10, part 2

Autocomplete tools are an odd bunch: the winner is found by sorting all of the scores and then taking the middle score. (There will always be an odd number of scores to consider.) In this example, the middle score is 288957 because there are the same number of scores smaller and larger than it.

Find the completion string for each incomplete line, score the completion strings, and sort the scores. What is the middle score?

In [9]:
def find_completion_sequence(line):
    stack = []
    pair_map = {
        '{': '}',
        '(': ')',
        '[': ']',
        '<': '>', 
    }
    close_map = {v: n for (n, v) in pair_map.items()}
    openings = set(pair_map.keys())
    endings = set(pair_map.values())
    
    for char in line:
        if char in openings:
            stack.append(char)
        
        if char in endings:
            if not stack:
                return []
            opening = stack.pop()
            expected = pair_map[opening]
            if char != expected:
                return []
        
    return [pair_map[char] for char in reversed(stack)]

In [10]:
def play_part_2(input):
    line_scores = []
    scores = {
        ')': 1,
        ']': 2,
        '}': 3,
        '>': 4
    }
    for line in input:
        line_score = 0
        completion_sequence = find_completion_sequence(line)
        for char in completion_sequence:
            line_score *= 5
            line_score += scores[char]
        
        if line_score:
            line_scores.append(line_score)
    
    line_scores = sorted(line_scores)
    median = line_scores[len(line_scores) // 2]
    return median

In [11]:
play_part_2(test_input), play_part_2(real_input)

(288957, 820045242)

### Day 10 Observations

It's frustrating when you know how to do part two but you've really got to go and catch a train. It would have been more elegant to represent the syntax errors as exceptions and so using the same function for parts 1 and 2.

## Day 11 Dumbo Octopus

https://adventofcode.com/2021/day/11

After 100 steps, there have been a total of 1656 flashes.

Given the starting energy levels of the dumbo octopuses in your cavern, simulate 100 steps. How many total flashes are there after 100 steps?

In [4]:
test_input = [
    '5483143223',
    '2745854711',
    '5264556173',
    '6141336146',
    '6357385478',
    '4167524645',
    '2176841721',
    '6882881134',
    '4846848554',
    '5283751526',
]

real_input = [
    '4836484555',
    '4663841772',
    '3512484556',
    '1481547572',
    '7741183422',
    '8683222882',
    '4215244233',
    '1544712171',
    '5725855786',
    '1717382281',
]

In [5]:
@dataclass(frozen=True)
class Point:
    x: int
    y: int
    
    @property
    def adjacent_points(self):
        return [
            Point(self.x - 1, self.y),
            Point(self.x + 1, self.y),
            Point(self.x, self.y + 1),
            Point(self.x, self.y - 1),
            Point(self.x - 1, self.y - 1),
            Point(self.x - 1, self.y + 1),
            Point(self.x + 1, self.y - 1),
            Point(self.x + 1, self.y + 1),
        ]


In [6]:
class Grid:
    def __init__(self, puzzle_input):
        self.points = {}
        self.max_x = len(puzzle_input[0]) - 1
        self.max_y = len(puzzle_input) - 1

        for y, line in enumerate(puzzle_input):
            for x, val in enumerate(line):
                self.points[Point(x, y)] = int(val)
    
    def get_adjacent_points(self, point):
            adjacent_points = point.adjacent_points
            adjacent_points = [
                point
                for point in adjacent_points
                if point.x >= 0 and point.x <= self.max_x
            ]
            adjacent_points = [
                point
                for point in adjacent_points
                if point.y >= 0 and point.y <= self.max_y
            ]
            return adjacent_points
    
    def display(self):
        for y in range(self.max_y + 1):
            for x in range(self.max_x + 1):
                print(self.points[Point(x, y)], end='')
            print()
            
    def incr_point(self, point, start_of_step):
        '''Returns whether or not it flashed'''
        val = self.points[point]
        if val == 0 and not start_of_step:
            return False

        val += 1
        val %= 10
        self.points[point] = val
        return val == 0
        
    def step(self):
        num_flashes = 0
        flashed = set()
        # increment by 1
        for point in self.points.keys():
            did_flash = self.incr_point(point, start_of_step=True)
            if did_flash:
                num_flashes += 1
                flashed.add(point)

        # Increment those around a flash
        while flashed:
            new_flashes = set()
            for point in flashed:
                adjacent_points = self.get_adjacent_points(point)
                for adjacent in adjacent_points:
                    did_flash = self.incr_point(adjacent, start_of_step=False)
                    if did_flash:
                        num_flashes += 1
                        new_flashes.add(adjacent)
            flashed = new_flashes
        
        return num_flashes

In [7]:
test_grid = Grid(test_input)
test_grid.display()
num_flashes = test_grid.step()
print()
test_grid.display()
num_flashes = test_grid.step()
print()
test_grid.display()
num_flashes

5483143223
2745854711
5264556173
6141336146
6357385478
4167524645
2176841721
6882881134
4846848554
5283751526

6594254334
3856965822
6375667284
7252447257
7468496589
5278635756
3287952832
7993992245
5957959665
6394862637

8807476555
5089087054
8597889608
8485769600
8700908800
6600088989
6800005943
0000007456
9000000876
8700006848


35

In [8]:
test_grid = Grid(test_input)
num_flashes = 0
for _ in range(100):
    num_flashes += test_grid.step()

num_flashes

1656

In [9]:
real_grid = Grid(real_input)
num_flashes = 0
for _ in range(100):
    num_flashes += real_grid.step()

num_flashes

1659

### Day 11, part 2

If you can calculate the exact moments when the octopuses will all flash simultaneously, you should be able to navigate through the cavern. What is the first step during which all octopuses flash?

In [10]:
test_grid = Grid(test_input)
for x in range(1000):
    num_flashes = test_grid.step()
    if num_flashes == 100:
        print(x + 1)
        break

195


In [11]:
real_grid = Grid(real_input)
for x in range(1000):
    num_flashes = real_grid.step()
    if num_flashes == 100:
        print(x + 1)
        break

227


### Day 11 Observations

Would be interesting to know how the puzzle inputs are generated. Is it possible to generate an input that never result in an all zero pattern?  It's not as if you can start from all zeros and generate a loop as the next states are all trivial (all ones, all twos etc).

## Day 12: Passage Pathing

https://adventofcode.com/2021/day/12

With your submarine's subterranean subsystems subsisting suboptimally, the only way you're getting out of this cave anytime soon is by finding a path yourself. Not just a path - the only way to know if you've found the best path is to find all of them.

...

So, the above cave system looks roughly like this:

```
    start
    /   \
c--A-----b--d
    \   /
     end
```

How many paths through this cave system are there that visit small caves at most once?

In [7]:
test_1 = {
    'start-A',
    'start-b',
    'A-c',
    'A-b',
    'b-d',
    'A-end',
    'b-end',
}

test_2 = {
    'dc-end',
    'HN-start',
    'start-kj',
    'dc-start',
    'dc-HN',
    'LN-dc',
    'HN-end',
    'kj-sa',
    'kj-HN',
    'kj-dc',
}

real_input = {
    'zs-WO',
    'zs-QJ',
    'WO-zt',
    'zs-DP',
    'WO-end',
    'gv-zt',
    'iu-SK',
    'HW-zs',
    'iu-WO',
    'gv-WO',
    'gv-start',
    'gv-DP',
    'start-WO',
    'HW-zt',
    'iu-HW',
    'gv-HW',
    'zs-SK',
    'HW-end',
    'zs-end',
    'DP-by',
    'DP-iu',
    'zt-start',
}

In [8]:
@dataclass(frozen=True)
class Cavern:
    name: str
    
    @property
    def is_big(self):
        return self.name == self.name.upper()

In [9]:
assert Cavern('AA').is_big is True
assert Cavern('aa').is_big is False
assert Cavern('start').is_big is False

In [10]:
def map_cave(puzzle_input):
    cave_map = defaultdict(set)
    for line in puzzle_input:
        cavern_1, cavern_2 = [
            Cavern(name)
            for name in line.split('-')
        ]
        cave_map[cavern_1].add(cavern_2)
        cave_map[cavern_2].add(cavern_1)
    
    return dict(cave_map)


In [11]:
map_cave(test_1)

{Cavern(name='start'): {Cavern(name='A'), Cavern(name='b')},
 Cavern(name='A'): {Cavern(name='b'),
  Cavern(name='c'),
  Cavern(name='end'),
  Cavern(name='start')},
 Cavern(name='c'): {Cavern(name='A')},
 Cavern(name='b'): {Cavern(name='A'),
  Cavern(name='d'),
  Cavern(name='end'),
  Cavern(name='start')},
 Cavern(name='end'): {Cavern(name='A'), Cavern(name='b')},
 Cavern(name='d'): {Cavern(name='b')}}

In [12]:
class BreadthFirstSolver(object):
    """Breadth first solver for game definitions."""

    def show_progress(self, num_moves, num_states):
        """Show progress whilst running."""
        print("Move {}: examining {} position{}.".format(
            num_moves,
            num_states,
            "s" if num_states == 1 else ""
        ))

    def solve(self, game, max_moves=0, find_all_solutions=False):
        """Solve the game returning a set of shortest solutions."""
        observed_states = set()
        # Set of tuples of (previous_moves_tuple, game_state)
        move_states = {((), game.initial_state)}
        num_moves = 0
        best_state = game.initial_state
        best_score = game.score(best_state)
        all_solutions = set()
        
        while True:
            # self.show_progress(num_moves, len(move_states))
            if max_moves and num_moves > max_moves:
                raise BufferError

            # Check for solutions
            solutions = {m for m, s in move_states if game.is_end_state(s)}
            if solutions:
                if not find_all_solutions:
                    return solutions
                
                all_solutions |= solutions

            # Check if we've found a better state:
            for _, s in move_states:
                if game.score(s) > best_score:
                    best_state = s
                    best_score = game.score(s)
                    print('High Score: ', best_score)
                
                # Hacked in to solve puzzle
                if num_moves == 40:
                    print ("Move 40: ", game.score(s)) 

            # Find the next level of states
            move_states = {
                (prev_moves + (move,), game.next_state(state, move))
                for (prev_moves, state) in move_states
                for move in game.next_moves(state)
            }

            # TODO - remove duplicates for unordered games

            # Prune and update previously observed states
            # (must have been part of a shorter solution)
            move_states = {
                (moves, state) for (moves, state) in move_states
                if state not in observed_states
            }
            if not move_states:
                #No possible next states == no more possible solutions
                return all_solutions
            observed_states.update([s for _, s in move_states])
            num_moves += 1

In [13]:
"""Interface for game definitions."""
class Game(object):

    @property
    def initial_state(self):
        """The initial state of the game.
        
        States must be hashable.
        """
        raise NotImplementedError()

    @property
    def ordered_moves(self):
        """Do the moves in the game have an order?
        Is [move 1, move2, move3] != [move2, move3, move1]
        """
        return True

    def next_moves(self, state):
        """Possible set of next moves from this `state`.
        If you're interested in the shortest solution only apply a heuristic
        if you _know_ that other moves cannot lead to a shorter solution. For
        example in the Tube Map Name problem it is fine to suggest 'Belsize Park'
        as a move given it's the only one with a 'Z' in it; hence must be in the
        shortest solution.
        """
        raise NotImplementedError()

    def next_state(self, state, move):
        """Next state of the system given move `move` in state `state`."""
        raise NotImplementedError()

    def is_end_state(self, state):
        """Is the current state `state` an end state of the game."""
        raise NotImplementedError()

    def score(self, state):
        """The score of the current `state`."""
        return 0

In [14]:
class Cave(Game):
    ordered_moves = True
    initial_state = (Cavern('start'), )

    def __init__(self, puzzle_input):
        self.cave_map = map_cave(puzzle_input)
        
    def next_moves(self, state):
        current_cavern = state[-1]
        small_caverns_visited = {cavern for cavern in state if not cavern.is_big}
        possible_moves = self.cave_map[current_cavern] - small_caverns_visited
        return possible_moves
    
    def next_state(self, state, move):
        return state + (move, )
    
    def is_end_state(self, state):
        return Cavern('end') == state[-1]

In [15]:
bfs = BreadthFirstSolver()
test_cave = Cave(test_1)
solutions = bfs.solve(test_cave, find_all_solutions=True)
len(solutions)

10

In [16]:
bfs = BreadthFirstSolver()
test_cave = Cave(test_2)
solutions = bfs.solve(test_cave, find_all_solutions=True)
len(solutions)

19

In [17]:
bfs = BreadthFirstSolver()
real_cave = Cave(real_input)
solutions = bfs.solve(real_cave, find_all_solutions=True)
len(solutions)

3738

In [18]:
class CavePart2(Cave):    
    def next_moves(self, state):
        current_cavern = state[-1]
        
        if Cavern('end') in state:
            return set()

        # Visited small caverns
        small_caverns_visited = [cavern for cavern in state if not cavern.is_big]
        small_counter = Counter(small_caverns_visited)
        revisited_small = bool([n for (n, v) in small_counter.items() if v >= 2])
        
        possible_moves = set()
        
        for move in self.cave_map[current_cavern]: 
            if move == Cavern('start'):
                continue

            if (
                move.is_big or
                not revisited_small or
                move not in small_counter.keys()
            ):
                possible_moves.add(move)

        return possible_moves

### Day 12, part 2

After reviewing the available paths, you realize you might have time to visit a single small cave twice. Specifically, big caves can be visited any number of times, a single small cave can be visited at most twice, and the remaining small caves can be visited at most once. However, the caves named start and end can only be visited exactly once each: once you leave the start cave, you may not return to it, and once you reach the end cave, the path must end immediately.

In [19]:
bfs = BreadthFirstSolver()
test_cave = CavePart2(test_1)
solutions = bfs.solve(test_cave, find_all_solutions=True)
len(solutions)

36

In [20]:
bfs = BreadthFirstSolver()
real_cave = CavePart2(real_input)
solutions = bfs.solve(real_cave, find_all_solutions=True)
len(solutions)

120506

### Day 12 Observations

So here's the maze that has been hinted at over the last few days. I managed to brush off and tweak and BFS solver I'd written for a previous year. It was slightly modified to enumerate all the possibilities rather than stopping at the shortest. A Depth First approach would have presumably been more efficient memory wise, given it only need concern itself with the current state, but I didn't have one of those to hand!

The second part is a classic case or - READ THE QUESTION. I read it (and re-read it several times the same way) as every small cavern could be visited twice. I immediately jump to the conclusion that the puzzle setter has made a mistake - how could there possibly be a bug in my code?



### Day 12 Revisited

Felt impure not solving this 'depth first', so adding a Depth First Solver to the tool box.

In [21]:
DepthFirstState = namedtuple(
    'DepthFirstState',
    ('state', 'untried_moves', 'move_history')
)

class DepthFirstSolver(object):
    """Breadth first solver for game definitions."""

    def solve(self, game):
        initial_state = game.initial_state
        initial_moves = game.next_moves(initial_state)
        
        state_stack = [
            DepthFirstState(
                state=game.initial_state,
                untried_moves=list(initial_moves),
                move_history=[]
            )
        ]
        solutions = set()
        
        while state_stack:
            state = state_stack.pop()
            if not state.untried_moves:
                continue
            
            move = state.untried_moves.pop()
            next_state = game.next_state(state.state, move)
            next_moves = game.next_moves(next_state)
            next_move_history = state.move_history + [move]
            state_stack.append(state)

            if game.is_end_state(next_state):
                solutions.add(tuple(next_move_history))
                continue
            
            state_stack.append(
                DepthFirstState(
                    state=next_state,
                    untried_moves=list(next_moves),
                    move_history=next_move_history
                )
            )
        
        return solutions

In [22]:
%%time

dfs = DepthFirstSolver()
real_cave = CavePart2(real_input)
solutions = dfs.solve(real_cave)
len(solutions)

CPU times: user 3.36 s, sys: 11.1 ms, total: 3.37 s
Wall time: 3.37 s


120506

In [23]:
%%time

bfs = BreadthFirstSolver()
real_cave = CavePart2(real_input)
solutions = bfs.solve(real_cave, find_all_solutions=True)
len(solutions)

CPU times: user 7.48 s, sys: 21.3 ms, total: 7.5 s
Wall time: 7.5 s


120506

## Day 13 Transparent Origami

https://adventofcode.com/2021/day/13

The transparent paper is pretty big, so for now, focus on just completing the first fold. After the first fold in the example above, 17 dots are visible - dots that end up overlapping after the fold is completed count as a single dot.

How many dots are visible after completing just the first fold instruction on your transparent paper?
    

In [3]:
test_input = get_puzzle_input('test_input_13.txt')
real_input = get_puzzle_input('input_13.txt')

In [4]:
@dataclass(frozen=True)
class Line:
    origin_offset: int
    is_horizontal: bool

@dataclass(frozen=True)
class Point:
    x: int
    y: int
    
    def fold(self, line):
        if (
            line.is_horizontal and line.origin_offset >= self.x or
            not line.is_horizontal and line.origin_offset >= self.y
        ):
            return self
        
        if line.is_horizontal:
            return Point(2*line.origin_offset - self.x, self.y)
        
        return Point(self.x, 2*line.origin_offset - self.y)

In [5]:
assert Point(10, 11).fold(Line(9, True)) == Point(8, 11)
assert Point(3, 11).fold(Line(9, True)) == Point(3, 11)
assert Point(10, 11).fold(Line(9, False)) == Point(10, 7)
assert Point(10, 7).fold(Line(9, False)) == Point(10, 7)

In [6]:
class Grid:
    fold_re = re.compile('fold along (.)=([0-9]+)')

    def __init__(self, puzzle_input):
        self.parse_input(puzzle_input)
    
    def parse_input(self, puzzle_input):
        self.points = set()
        self.fold_lines = []
        
        for line in puzzle_input:
            components = line.split(',')
            if len(components) == 2:
                point = Point(int(components[0]), int(components[1]))
                self.points.add(point)
                continue
            
            match = self.fold_re.match(line)
            
            if match:
                horizontal = match.groups()[0] == 'x'
                origin_offset = int(match.groups()[1])
                self.fold_lines.append(Line(origin_offset, horizontal))
    
    def fold(self, fold_line):
        new_points = {point.fold(fold_line) for point in self.points}
        self.points = new_points

    def display(self):
        max_x = max(point.x for point in self.points)
        max_y = max(point.y for point in self.points)
        
        for y in range(max_y + 1):
            for x in range(max_x + 1):
                char = '#' if Point(x, y) in self.points else '.'
                print(char, end='')
            print()
       

In [7]:
def part_1(puzzle_input):
    grid = Grid(puzzle_input)
    grid.fold(grid.fold_lines[0])
    return len(grid.points)

In [8]:
part_1(test_input), part_1(real_input)

(17, 781)

### Day 13, part 2

Finish folding the transparent paper according to the instructions. The manual says the code is always eight capital letters.

What code do you use to activate the infrared thermal imaging camera system?

In [9]:
# Ensure no stupid mistakes in display
test_grid = Grid(test_input)
test_grid.display()

...#..#..#.
....#......
...........
#..........
...#....#.#
...........
...........
...........
...........
...........
.#....#.##.
....#......
......#...#
#..........
#.#........


In [10]:
real_grid = Grid(real_input)
for line in real_grid.fold_lines:
    real_grid.fold(line)
real_grid.display()

###..####.###...##...##....##.###..###.
#..#.#....#..#.#..#.#..#....#.#..#.#..#
#..#.###..#..#.#....#.......#.#..#.###.
###..#....###..#....#.##....#.###..#..#
#....#....#.#..#..#.#..#.#..#.#....#..#
#....####.#..#..##...###..##..#....###.


### Day 13 Observations

This would be a good example of what I like in a puzzle - elegant, but not necessarily (and preferably not) difficult. Often elegant problems have the characteristic of being much harder to set than to solve.

## Day 14 Extended Polymerization

https://adventofcode.com/2021/day/14

Apply 10 steps of pair insertion to the polymer template and find the most and least common elements in the result. What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?

In [1]:
test_start = list('NNCB')

test_element_map = {
    'CH': 'B',
    'HH': 'N',
    'CB': 'H',
    'NH': 'C',
    'HB': 'C',
    'HC': 'B',
    'HN': 'C',
    'NN': 'C',
    'BH': 'H',
    'NC': 'B',
    'NB': 'B',
    'BN': 'B',
    'BB': 'N',
    'BC': 'B',
    'CC': 'N',
    'CN': 'C',
}

In [2]:
real_start = list('OFSVVSFOCBNONHKFHNPK')

real_element_map = {
    'HN': 'C',
    'VB': 'K',
    'PF': 'C',
    'BO': 'F',
    'PB': 'F',
    'OH': 'H',
    'OB': 'N',
    'PN': 'O',
    'KO': 'V',
    'CK': 'V',
    'FP': 'H',
    'PC': 'V',
    'PP': 'N',
    'FN': 'N',
    'CC': 'F',
    'FC': 'N',
    'BP': 'N',
    'SH': 'F',
    'NS': 'V',
    'KK': 'B',
    'HS': 'C',
    'NV': 'N',
    'FO': 'B',
    'VO': 'S',
    'KN': 'F',
    'SC': 'V',
    'NB': 'H',
    'CH': 'B',
    'SF': 'V',
    'NP': 'V',
    'FB': 'P',
    'CV': 'B',
    'PO': 'P',
    'SV': 'P',
    'OO': 'V',
    'PS': 'C',
    'CO': 'N',
    'SP': 'B',
    'KP': 'H',
    'KH': 'S',
    'KS': 'S',
    'NH': 'K',
    'SS': 'P',
    'PV': 'P',
    'KV': 'V',
    'ON': 'N',
    'BS': 'C',
    'HP': 'K',
    'SB': 'P',
    'VC': 'B',
    'HB': 'N',
    'FS': 'V',
    'VP': 'K',
    'BB': 'N',
    'FK': 'S',
    'CS': 'P',
    'SO': 'F',
    'HF': 'F',
    'VV': 'C',
    'BC': 'S',
    'SN': 'K',
    'KB': 'H',
    'BN': 'H',
    'HO': 'S',
    'KC': 'F',
    'CP': 'S',
    'HC': 'S',
    'OS': 'K',
    'NK': 'N',
    'BF': 'S',
    'VN': 'B',
    'SK': 'K',
    'HV': 'B',
    'KF': 'H',
    'FV': 'B',
    'VF': 'H',
    'BH': 'S',
    'NN': 'O',
    'HH': 'K',
    'CN': 'H',
    'PH': 'V',
    'NF': 'S',
    'OV': 'P',
    'OC': 'V',
    'OK': 'H',
    'OF': 'H',
    'HK': 'N',
    'FH': 'P',
    'BK': 'N',
    'VS': 'H',
    'NO': 'V',
    'VK': 'K',
    'CF': 'N',
    'CB': 'N',
    'NC': 'K',
    'PK': 'B',
    'VH': 'F',
    'FF': 'C',
    'BV': 'P',
    'OP': 'K',
}

In [4]:
# Use the windowing function from day 1
def get_pairs(state):
    pairs = windows(state, 2)
    # lose the spare
    pairs.pop()
    return pairs

In [5]:
def get_pair_map(string_map):
    return {
        tuple(n): list([tuple(n)[0], v, tuple(n)[1]])
        for n, v in string_map.items()
    }

In [6]:
get_pair_map(test_element_map)

{('C', 'H'): ['C', 'B', 'H'],
 ('H', 'H'): ['H', 'N', 'H'],
 ('C', 'B'): ['C', 'H', 'B'],
 ('N', 'H'): ['N', 'C', 'H'],
 ('H', 'B'): ['H', 'C', 'B'],
 ('H', 'C'): ['H', 'B', 'C'],
 ('H', 'N'): ['H', 'C', 'N'],
 ('N', 'N'): ['N', 'C', 'N'],
 ('B', 'H'): ['B', 'H', 'H'],
 ('N', 'C'): ['N', 'B', 'C'],
 ('N', 'B'): ['N', 'B', 'B'],
 ('B', 'N'): ['B', 'B', 'N'],
 ('B', 'B'): ['B', 'N', 'B'],
 ('B', 'C'): ['B', 'B', 'C'],
 ('C', 'C'): ['C', 'N', 'C'],
 ('C', 'N'): ['C', 'C', 'N']}

In [7]:
def get_next_state(state, pair_map):
    get_pairs(state)
    triples = [pair_map[pair] for pair in get_pairs(state)]
    # Remove the duplicated state
    next_state = list(triples[0])
    for t in triples[1:]:
        next_state.extend(t[1:])
        
    return next_state

In [8]:
def part_1(start, element_map, steps):
    pair_map = get_pair_map(element_map)
    state = start
    for step in range(steps):
        state = get_next_state(state, pair_map)

    return ''.join(state)

In [9]:
assert part_1(test_start, test_element_map, 1) == 'NCNBCHB'
assert part_1(test_start, test_element_map, 2) == 'NBCCNBBBCBHCB'
assert part_1(test_start, test_element_map, 3) == 'NBBBCNCCNBBNBNBBCHBHHBCHB'
assert part_1(test_start, test_element_map, 4) == 'NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB'

In [12]:
state_10 = part_1(test_start, test_element_map, 10)
counter = Counter(state_10)
counter.most_common()

[('B', 1749), ('N', 865), ('C', 298), ('H', 161)]

In [20]:
state_10 = part_1(real_start, real_element_map, 10)
counter = Counter(state_10)
counter.most_common()

[('N', 4285),
 ('P', 2777),
 ('V', 2050),
 ('S', 1943),
 ('H', 1797),
 ('O', 1596),
 ('C', 1539),
 ('F', 1318),
 ('B', 1151),
 ('K', 1001)]

In [21]:
4285 - 1001

3284

### Day 14: Part 2

Apply 40 steps of pair insertion to the polymer template and find the most and least common elements in the result. What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?

In [13]:
%%time

# Hmmm big numbers - previous solution ain't gonna fly
# Let's get a feel for the pain
len(part_1(real_start, real_element_map, 20))

CPU times: user 15.3 s, sys: 471 ms, total: 15.7 s
Wall time: 15.7 s


19922945

OK - plan B...

In [14]:
def make_compound_map(element_map):
    return {
        n: [f'{n[0]}{v}', f'{v}{n[1]}']
        for n, v in element_map.items()
    }

In [15]:
make_compound_map(test_element_map)

{'CH': ['CB', 'BH'],
 'HH': ['HN', 'NH'],
 'CB': ['CH', 'HB'],
 'NH': ['NC', 'CH'],
 'HB': ['HC', 'CB'],
 'HC': ['HB', 'BC'],
 'HN': ['HC', 'CN'],
 'NN': ['NC', 'CN'],
 'BH': ['BH', 'HH'],
 'NC': ['NB', 'BC'],
 'NB': ['NB', 'BB'],
 'BN': ['BB', 'BN'],
 'BB': ['BN', 'NB'],
 'BC': ['BB', 'BC'],
 'CC': ['CN', 'NC'],
 'CN': ['CC', 'CN']}

In [16]:
def part_2(start_state, element_map, steps):
    '''It doesn't matter where the compound occurs in the chain
    
    ABC -> A+f(AB) + f(AB)+B + B+f(BC) + f(BC)+C
    The first part only depends on AB and the second only depends on BC.
    So I can treat them independently:
    AB -> A+f(AB) + f(AB)+B
    BC -> B+f(BC) + f(BC)+C
    
    One molecule of NN always produces one molecule of NC and one of CN,
    so 3NN -> 3NC + 3CN
    '''
    element_counter = Counter(start_state)
    compound_counter = Counter([''.join(pair) for pair in get_pairs(start_state)])
    compound_map = make_compound_map(element_map)

    for step in range(steps):
        for compound, num in list(compound_counter.items()):
            new_element = element_map[compound]
            element_counter[new_element] += num
            compound_counter[compound] -= num
            
            for new_compound in compound_map[compound]:
                compound_counter[new_compound] += num

    return element_counter

In [17]:
part_2(test_start, test_element_map, 40).most_common()

[('B', 2192039569602),
 ('N', 1096047802353),
 ('C', 6597635301),
 ('H', 3849876073)]

In [18]:
part_2(real_start, real_element_map, 40).most_common()

[('N', 5224941105516),
 ('P', 2635945447382),
 ('V', 2299932080861),
 ('O', 2202794261121),
 ('S', 1915040933128),
 ('H', 1753226545958),
 ('F', 1471801718164),
 ('C', 1381228953516),
 ('B', 1083544306272),
 ('K', 922265575827)]

In [19]:
5224941105516 - 922265575827

4302675529689

### Day 14 Observations

I spent a stupid amount of time debugging the first part of this where I had a bug that passed the reference to a list rather than a copy. It produced the correct output for step 1 and 2 but went crazy thereafter and wouldn't return past five steps - not a great start to part 2. For part 2 I needed to resort to power tools - pen and paper - to figure out that the joining (AB + BC -> ABC) and splitting operations (AB -> A + B) effectively cancel one another out. I can see it but I still have trouble trying to explain it to myself. Nice puzzle.

## Day 15 Chiton

https://adventofcode.com/2021/day/15

The total risk of this path is 40 (the starting position is never entered, so its risk is not counted).

What is the lowest total risk of any path from the top left to the bottom right?

In [4]:
# Going to use Dijkstra and not going to reinvent the wheel
from dijkstar import Graph, find_path

In [5]:
test_input = get_puzzle_input('test_input_15.txt')
real_input = get_puzzle_input('input_15.txt')

In [6]:
# Borrowed from day 9
@dataclass(frozen=True)
class Point:
    x: int
    y: int
    
    @property
    def adjacent_points(self):
        return [
            Point(self.x - 1, self.y),
            Point(self.x + 1, self.y),
            Point(self.x, self.y + 1),
            Point(self.x, self.y - 1),
        ]

In [7]:
class Grid:
    def __init__(self, puzzle_input, extend_grid=False):
        self.points = {}
        self.max_x = len(puzzle_input[0]) - 1
        self.max_y = len(puzzle_input) - 1

        for y, line in enumerate(puzzle_input):
            for x, val in enumerate(line):
                self.points[Point(x, y)] = int(val)
        
        if extend_grid:
            self.extend_grid()
        
        self.make_graph()
    
    def extend_grid(self):
        """Part 2"""
        extended_points = dict(self.points)
        for y in range(0, 5):
            for x in range(0, 5):
                if x == 0 and y == 0:
                    continue
                for point in self.points:
                    new_point = Point(
                        point.x + (self.max_x + 1) * x,
                        point.y + (self.max_y + 1) * y
                    )
                    new_val = (self.points[point] + x + y - 1) % 9 + 1
                    extended_points[new_point] = new_val
            
        self.points = extended_points
        self.max_x = 5 * (self.max_x + 1) - 1
        self.max_y = 5 * (self.max_y + 1) - 1
    
    def display(self):
        for y in range(self.max_y + 1):
            for x in range(self.max_x + 1):
                print(self.points[Point(x, y)], end='')
            print()
                    
    def get_adjacent_points(self, point):
            adjacent_points = point.adjacent_points
            adjacent_points = [
                point
                for point in adjacent_points
                if point.x >= 0 and point.x <= self.max_x
            ]
            adjacent_points = [
                point
                for point in adjacent_points
                if point.y >= 0 and point.y <= self.max_y
            ]
            return adjacent_points
    
    def make_graph(self):
        self.graph = Graph()
        for point in self.points:
            for neighbour in self.get_adjacent_points(point):
                self.graph.add_edge(point, neighbour, self.points[neighbour])


In [8]:
test_grid = Grid(test_input)
path = find_path(test_grid.graph, Point(0,0), Point(test_grid.max_x, test_grid.max_y))
path.total_cost

40

In [9]:
real_grid = Grid(real_input)
path = find_path(real_grid.graph, Point(0,0), Point(real_grid.max_x, real_grid.max_y))
path.total_cost

462

### Day 15, Part 2

Using the full map, what is the lowest total risk of any path from the top left to the bottom right?

In [11]:
test_grid = Grid(test_input, extend_grid=True)
test_grid.display()

11637517422274862853338597396444961841755517295286
13813736722492484783351359589446246169155735727126
21365113283247622439435873354154698446526571955763
36949315694715142671582625378269373648937148475914
74634171118574528222968563933317967414442817852555
13191281372421239248353234135946434524615754563572
13599124212461123532357223464346833457545794456865
31254216394236532741534764385264587549637569865174
12931385212314249632342535174345364628545647573965
23119445813422155692453326671356443778246755488935
22748628533385973964449618417555172952866628316397
24924847833513595894462461691557357271266846838237
32476224394358733541546984465265719557637682166874
47151426715826253782693736489371484759148259586125
85745282229685639333179674144428178525553928963666
24212392483532341359464345246157545635726865674683
24611235323572234643468334575457944568656815567976
42365327415347643852645875496375698651748671976285
23142496323425351743453646285456475739656758684176
3422155692453326671356443778246

In [12]:
path = find_path(test_grid.graph, Point(0,0), Point(test_grid.max_x, test_grid.max_y))
path.total_cost

315

In [13]:
real_grid = Grid(real_input, extend_grid=True)
path = find_path(real_grid.graph, Point(0,0), Point(real_grid.max_x, real_grid.max_y))
path.total_cost

2846

### Day 15 Observations

This is one of those puzzle that if you know the algorithms it falls straight out otherwise it's an awful struggle. I could have supplied the manhattan distance as the A* heuristic but everything returns quick enough for me as is.

## Day 16 Packet Decoder

https://adventofcode.com/2021/day/16

Decode the structure of your hexadecimal-encoded BITS transmission; what do you get if you add up the version numbers in all packets?

In [19]:
real_input = '220D4B80491FE6FBDCDA61F23F1D9B763004A7C128012F9DA88CE27B000B30F4804D49CD515380352100763DC5E8EC000844338B10B667A1E60094B7BE8D600ACE774DF39DD364979F67A9AC0D1802B2A41401354F6BF1DC0627B15EC5CCC01694F5BABFC00964E93C95CF080263F0046741A740A76B704300824926693274BE7CC880267D00464852484A5F74520005D65A1EAD2334A700BA4EA41256E4BBBD8DC0999FC3A97286C20164B4FF14A93FD2947494E683E752E49B2737DF7C4080181973496509A5B9A8D37B7C300434016920D9EAEF16AEC0A4AB7DF5B1C01C933B9AAF19E1818027A00A80021F1FA0E43400043E174638572B984B066401D3E802735A4A9ECE371789685AB3E0E800725333EFFBB4B8D131A9F39ED413A1720058F339EE32052D48EC4E5EC3A6006CC2B4BE6FF3F40017A0E4D522226009CA676A7600980021F1921446700042A23C368B713CC015E007324A38DF30BB30533D001200F3E7AC33A00A4F73149558E7B98A4AACC402660803D1EA1045C1006E2CC668EC200F4568A5104802B7D004A53819327531FE607E118803B260F371D02CAEA3486050004EE3006A1E463858600F46D8531E08010987B1BE251002013445345C600B4F67617400D14F61867B39AA38018F8C05E430163C6004980126005B801CC0417080106005000CB4002D7A801AA0062007BC0019608018A004A002B880057CEF5604016827238DFDCC8048B9AF135802400087C32893120401C8D90463E280513D62991EE5CA543A6B75892CB639D503004F00353100662FC498AA00084C6485B1D25044C0139975D004A5EB5E52AC7233294006867F9EE6BA2115E47D7867458401424E354B36CDAFCAB34CBC2008BF2F2BA5CC646E57D4C62E41279E7F37961ACC015B005A5EFF884CBDFF10F9BFF438C014A007D67AE0529DED3901D9CD50B5C0108B13BAFD6070'

In [20]:
def padded_binary_list(num):
    '''Returns 0-15 as a 4 char list of `1`s and `0`s'''  
    return list("{0:04b}".format(num))
    

assert padded_binary_list(2) == ['0', '0', '1', '0']
assert padded_binary_list(15) == ['1', '1', '1', '1']

In [21]:
def to_binary_stream(hex_stream):
    '''Turns a stream of hex chars into a deque of 4 char bins'''
    stream = []
    for hex_num in hex_stream:
        stream.extend(padded_binary_list(int(hex_num, 16)))
    
    return deque(stream)

assert to_binary_stream('A1') == deque(['1', '0', '1', '0', '0', '0', '0', '1'])

In [22]:
def binary_pop(stream, num_digits):
    '''Take num_digits char bits from the stream'''
    result = []
    for _ in range(num_digits):
        result.append(stream.popleft())
    
    return int(''.join(result), 2)

stream = deque(['1', '0', '1', '0', '0', '0', '0', '1'])
assert binary_pop(stream, 3) == 5
assert binary_pop(stream, 5) == 1
assert stream == deque()

In [23]:
def stream_pop(stream, num_digits):
    '''Take the first num_digits from the deque and return as a new deque'''
    result = []
    for _ in range(num_digits):
        result.append(stream.popleft())
    
    return deque(result)

stream = deque(['1', '0', '1', '0', '0', '0', '0', '1'])
assert stream_pop(stream, 3) == deque(['1', '0', '1'])

In [24]:
class Packet:
    def consume(self, stream):
        raise NotImplementedError()

    @property
    def version_sum(self):
        raise NotImplementedError()
    
    def evaluate(self):
        raise NotImplementedError()

class PacketFactory:
    def build(self, stream):
        version = binary_pop(stream, 3)
        op_type = binary_pop(stream, 3)

        if op_type == 4:
            packet = LiteralPacket(version, 4)
        else:
            packet = OperatorPacket(version, op_type)
        
        packet.consume(stream)
        return packet
        
packet_factory = PacketFactory() 


In [25]:
class LiteralPacket(Packet):
    def __init__(self, version, op_type):
        self.version = version
        self.op_type = op_type

    def consume(self, stream):
        binary_list = []
        halt = False
        while not halt:
            figure = binary_pop(stream, 5)
            if not figure & 16:
                halt = True
            figure = figure & 15
            binary_list.extend(padded_binary_list(figure))
    
        self.value = int(''.join(binary_list), 2)

    @property
    def version_sum(self):
        return self.version

    def evaluate(self):
        return self.value
    

stream = deque('10101101000010000')
literal = LiteralPacket(100, 4)
literal.consume(stream)
assert literal.value == 1348
assert literal.version_sum == 100
assert stream == deque(['0', '0'])

stream = deque('101111111000101000')
literal = LiteralPacket(3, 4)
literal.consume(stream)
assert literal.value == 2021
assert literal.version_sum == 3

In [26]:
class OperatorPacket(Packet):
    def __init__(self, version, op_type):
        self.version = version
        self.op_type = op_type
        self.operands = []
        self.num_operands = None

    @property
    def operands_required(self):
        if self.num_operands is None:
            return bool(self.stream)
        
        return len(self.operands) < self.num_operands

    def consume(self, stream):
        by_num_packets = binary_pop(stream, 1)
        if by_num_packets:
            self.num_operands = binary_pop(stream, 11)
            # Consume from the parent stream
            self.stream = stream
        else:
            operand_bits_remaining = binary_pop(stream, 15)
            # Grab our own portion of the stream to consume
            self.stream = stream_pop(stream, operand_bits_remaining)
        
        while self.operands_required:
            self.consume_operand()

    def consume_operand(self):
        operand = packet_factory.build(self.stream)
        self.operands.append(operand)

    @property
    def version_sum(self):
        return sum(operand.version_sum for operand in self.operands) + self.version

    def evaluate(self):
        gt = lambda x: int(x[0] > x[1])
        lt = lambda x: int(x[0] < x[1])
        eq = lambda x: int(x[0] == x[1])
        op_map = {
            0: sum,
            1: math.prod,
            2: min,
            3: max,
            5: gt,
            6: lt,
            7: eq
        }
        
        values = [operand.evaluate() for operand in self.operands]
        return op_map[self.op_type](values)


8A004A801A8002F478 represents an operator packet (version 4) which contains an operator packet (version 1) which contains an operator packet (version 5) which contains a literal value (version 6); this packet has a version sum of 16.


In [27]:
test_stream = to_binary_stream('8A004A801A8002F478')
packet = packet_factory.build(test_stream)
packet.version_sum

16

In [28]:
test_stream = to_binary_stream('A0016C880162017C3686B18A3D4780')
packet = packet_factory.build(test_stream)
packet.version_sum

31

In [29]:
real_stream = to_binary_stream(real_input)
packet = packet_factory.build(real_stream)
packet.version_sum

977

### Day 16, part 2

What do you get if you evaluate the expression represented by your hexadecimal-encoded BITS transmission?

In [30]:
test_stream = to_binary_stream('C200B40A82')
test_op = packet_factory.build(test_stream)
test_op.evaluate()

3

In [31]:
real_stream = to_binary_stream(real_input)
real_op = packet_factory.build(real_stream)
real_op.evaluate()

101501020883

### Day 16 Observations

That was cognitive overload for a Thursday morning. I'm done. Bit concerned about building on this in future days with all the packets having their grubby mits on the input stream.

### Day 16 Revisited

OK... off the back of thinking - uh oh, where are the operators 8 and 9 - forgetting that its encoded as 3 bits, I refactored the stream in case there were any sort of JMP operations coming up in the next couple of days. So it's now a ticker tape with a pointer rather than popping off a deque.

In [32]:
class BinaryStream:
    def __init__(self, stream, is_hex=True):
        self.position = 0
        if is_hex:
            self.from_hex(stream)
        else:
            self.stream = list(stream)

    @property
    def bits_remaining(self):
        return len(self.stream) - self.position

    def from_hex(self, hex_stream):
        '''Turns a stream of hex chars into a deque of 4 char bins'''
        self.stream = []
        for hex_num in hex_stream:
            self.stream.extend(padded_binary_list(int(hex_num, 16)))

    def read(self, num_bits, as_stream=False):
        start_pos = self.position
        end_pos = start_pos + num_bits
        
        values = self.stream[start_pos:end_pos]
        self.position += num_bits
        
        if as_stream:
            return BinaryStream(values, is_hex=False)
        
        return int(''.join(values), 2)

    def __repr__(self):
        stream_repr = hash(''.join(self.stream))
        return f'BinaryStream(st={stream_repr}, len={len(self.stream)}, pos={self.position})'
        
        
binary_stream = BinaryStream('A1')
assert binary_stream.stream == ['1', '0', '1', '0', '0', '0', '0', '1']

binary_stream = BinaryStream('10100001', is_hex=False)
assert binary_stream.stream == ['1', '0', '1', '0', '0', '0', '0', '1']

binary_stream = BinaryStream('10100001', is_hex=False)
assert binary_stream.read(3) == 5
assert binary_stream.read(5) == 1
assert binary_stream.bits_remaining == 0

binary_stream = BinaryStream('10100001', is_hex=False)
new_stream = binary_stream.read(3, as_stream=True)
assert new_stream.position == 0
assert new_stream.stream == list('101')

In [33]:
class Packet:
    def read(self, stream):
        raise NotImplementedError()

    @property
    def version_sum(self):
        raise NotImplementedError()
    
    def evaluate(self):
        raise NotImplementedError()

In [34]:
class PacketFactory:
    def __init__(self, logging=False):
        self.logging = logging

    def build(self, stream):
        if self.logging:
            print(f'Reading stream: {stream}')
            
        version = stream.read(3)
        op_type = stream.read(3)

        if op_type == 4:
            packet = LiteralPacket(version, 4)
        else:
            packet = OperatorPacket(version, op_type)
        
        if self.logging:
            print(f'New packet stream: {stream} version: {version}, op_type: {op_type}')

        packet.read(stream)

        return packet
        
packet_factory = PacketFactory(logging=False) 

In [35]:
class LiteralPacket(Packet):
    def __init__(self, version, op_type):
        self.version = version
        self.op_type = op_type

    def read(self, stream):
        binary_list = []
        halt = False
        while not halt:
            figure = stream.read(5)
            halt = not figure & 16
            figure = figure & 15
            binary_list.extend(padded_binary_list(figure))
    
        self.value = int(''.join(binary_list), 2)

    @property
    def version_sum(self):
        return self.version

    def evaluate(self):
        return self.value
    
    def __repr__(self):
        return str(self.value)

In [36]:
stream = BinaryStream('10101101000010000', is_hex=False)
literal = LiteralPacket(100, 4)
literal.read(stream)
assert literal.value == 1348
assert literal.version_sum == 100
assert stream.bits_remaining == 2

stream = BinaryStream('101111111000101000', is_hex=False)
literal = LiteralPacket(3, 4)
literal.read(stream)
assert literal.value == 2021
assert literal.version_sum == 3
assert stream.bits_remaining == 3

In [37]:
class OperatorPacket(Packet):
    def __init__(self, version, op_type):
        self.version = version
        self.op_type = op_type
        self.operands = []
        self.num_operands = None

    @property
    def operands_required(self):
        if self.num_operands is None:
            return bool(self.stream.bits_remaining)
        
        return len(self.operands) < self.num_operands

    def read(self, stream):
        by_num_packets = stream.read(1)
        if by_num_packets:
            self.num_operands = stream.read(11)
            # Consume from the parent stream
            self.stream = stream
        else:
            operand_bits_remaining = stream.read(15)
            # Grab our own portion of the stream to consume
            self.stream = stream.read(operand_bits_remaining, as_stream=True)
        
        while self.operands_required:
            self.consume_operand()

    def consume_operand(self):
        operand = packet_factory.build(self.stream)
        self.operands.append(operand)

    @property
    def version_sum(self):
        return sum(operand.version_sum for operand in self.operands) + self.version

    def evaluate(self):
        gt = lambda x: int(x[0] > x[1])
        lt = lambda x: int(x[0] < x[1])
        eq = lambda x: int(x[0] == x[1])
        op_map = {
            0: sum,
            1: math.prod,
            2: min,
            3: max,
            5: gt,
            6: lt,
            7: eq
        }
        
        values = [operand.evaluate() for operand in self.operands]
        return op_map[self.op_type](values)

    def __repr__(self):
        op_str_map = {
            0: '+',
            1: '*',
            2: 'min',
            3: 'min',
            5: '>',
            6: '<',
            7: '=='
        }
        operand_repr = ', '.join([str(o) for o in self.operands])
        
        return f'({op_str_map[self.op_type]}, {operand_repr})'


```
8A004A801A8002F478 represents an operator packet (version 4) which contains an operator packet (version 1) which contains an operator packet (version 5) which contains a literal value (version 6); this packet has a version sum of 16.
```

In [38]:
test_stream = BinaryStream('8A004A801A8002F478')
packet = packet_factory.build(test_stream)
print(packet)
packet.version_sum

(min, (min, (min, 15)))


16

In [39]:
test_stream = BinaryStream('A0016C880162017C3686B18A3D4780')
packet = packet_factory.build(test_stream)
print(packet)
packet.version_sum

(+, (+, (+, 6, 6, 12, 15, 15)))


31

In [40]:
real_stream = BinaryStream(real_input)
packet = packet_factory.build(real_stream)
packet.version_sum

977

In [41]:
test_stream = BinaryStream('C200B40A82')
test_op = packet_factory.build(test_stream)
print(test_op)
test_op.evaluate()

(+, 1, 2)


3

In [42]:
real_stream = BinaryStream(real_input)
real_op = packet_factory.build(real_stream)
real_op.evaluate()

101501020883

And in case you were wondering (you weren't), here's the calculation:

```python
print(real_op)
```

```
(+, 
 (min, 15, 199071281, 499, 190),
 (*, 240, (>, 3410, 201)),
 (min, 7),
 (*, 51786, (==, (+, 15, 11, 14), (+, 6, 11, 5))),
 2513,
 (*, 28408, (>, 52653, 52653)),
 751,
 936579,
 (*, 84, (>, 246356739, 15228190)),
 (*, 321243, (==, 212, 631)),
 (*, (==, 208, 208), 54640),
 (*, 41, 169, 213, 246),
 (*, (<, 34, 34), 31562),
 (+, 1625637177),
 (min, 852, 650304435, 1, 651481, 70),
 (+, 409369, 4146254, 19617, 217, 45),
 13360882,
 (*, 118, 106, 171), 
 10570687,
 (*, (<, 33, 3854067120), 2456541),
 (min, 55, 42444, 6),
 (*, (<, (+, 15, 3, 6), (+, 5, 12, 5)), 5833350),
 (*, (<, 2729, 15305503), 101077955),
 (+, 155057593, 13, 37107853833, 5),
 (min, 10082448, 410034, 1891),
 (*, 138),
 497655,
 (min, 3),
 42002,
 (*, 18874, (>, (+, 12, 5, 12), (+, 8, 6, 11))),
 9,
 (*, (==, 35175563, 133), 3584),
 (*, 3683, (<, 248005212, 248005212)),
 (*, (+, 15, 4, 11), (+, 11, 12, 14), (+, 10, 10, 8)),
 (*, (<, 1217, 151), 588931), 4,
 (+, 48, 45472, 94453890),
 (+, (*, 7, 12, 5), (*, 13, 5, 15), (*, 7, 13, 9)),
 (*, 164, 165),
 (*, 59825, (>, 3724, 956760)),
 (+, (min, (min, (*, (*, (*, (min, (min, (+, (*, (+, (+, (min, (min, (min, (+, (+, (min, (min, (min, 3547)))))))))))))))))))),
 (+, 2, 101374),
 (*, 2788, (>, (+, 6, 9, 9), (+, 13, 2, 15))),
 (min, 3685, 7582, 81, 701716, 429274),
 (*, (<, (+, 11, 9, 10), (+, 12, 5, 14)), 66),
 (*, 158, (<, 123061340, 309)),
 (min, 265210578, 10),
 31918501,
 (*, 205, 109, 187, 92, 159),
 (min, 29305, 2195, 98863220, 15518495373),
 (*, (>, 491281, 491281), 49105),
 (*, (>, 3696, 81611779), 51781),
 (min, 11, 14349575)
)
```

## Day 17 Trick Shot

https://adventofcode.com/2021/day/17

Find the initial velocity that causes the probe to reach the highest y position and still eventually be within the target area after any step. What is the highest y position it reaches on this trajectory?

In [2]:
@dataclass(frozen=True)
class Point:
    x: int
    y: int

@dataclass(frozen=True)
class TargetArea:
    min_x: int
    max_x: int
    min_y: int
    max_y: int
    
    def hit_x(self, x_pos):
        return x_pos >= self.min_x and x_pos <= self.max_x

    def hit_y(self, y_pos):
        return y_pos >= self.min_y and y_pos <= self.max_y
    
    def hit(self, point):
        return self.hit_x(point.x) and self.hit_y(point.y)

In [3]:
test_target_area = TargetArea(20, 30, -10, -5)
real_target_area = TargetArea(85, 145, -163, -108)

In [4]:
def y_pos(velocity, day):
    '''Y position on day `day` with initial y velocity component `velocity`
    
    = y + (y - 1) + ... (y - (n-1))
    = ny - (1 + 2 + (n-1))
    = ny - ((1 + 2 + n) - n)
    = n(y + 1) - n(n + 1) / 2 
    '''
    return int(day * (velocity + 1) - day * (day + 1) / 2)

def x_pos(velocity, day):
    '''X position on day `day` with initial x velocity component `velocity`
    
    same as y velocity calc until it stops
    '''
    
    stop_day = velocity + 1
    effective_day = stop_day if day >= stop_day else day
    return y_pos(velocity, effective_day)

def is_moving_horizontally(x_velocity, day):
    '''Are we still moving horizontally on day `day`
    
    If False it won't move the next day (may have moved today)
    '''
    return x_velocity > day

In [6]:
assert y_pos(2, 1) == 2
assert y_pos(2, 2) == 3
assert y_pos(2, 3) == 3
assert y_pos(2, 4) == 2
assert y_pos(2, 5) == 0
assert y_pos(2, 6) == -3
assert y_pos(2, 7) == -7

assert x_pos(2, 1) == 2
assert x_pos(2, 2) == 3
assert x_pos(2, 3) == 3
assert x_pos(2, 4) == 3

assert is_moving_horizontally(2, 1) is True
assert is_moving_horizontally(2, 2) is False
assert is_moving_horizontally(2, 3) is False
assert is_moving_horizontally(2, 2) is False

In [7]:
def find_possible_x_velocities(target_area):
    min_vel = 1
    max_vel = target_area.max_x
    valid = []
    
    for vel in range(min_vel, max_vel + 1):
        day = 1
        while True:
            pos = x_pos(vel, day)
            is_stopped = not is_moving_horizontally(vel, day)
            if target_area.hit_x(pos):
                valid.append((vel, day, is_stopped))

            overshoot = pos > target_area.max_x
            
            if is_stopped or overshoot:
                break
            
            day += 1
    
    return valid

In [8]:
find_possible_x_velocities(test_target_area)

[(6, 5, False),
 (6, 6, True),
 (7, 4, False),
 (7, 5, False),
 (7, 6, False),
 (7, 7, True),
 (8, 3, False),
 (8, 4, False),
 (8, 5, False),
 (9, 3, False),
 (9, 4, False),
 (10, 3, False),
 (11, 2, False),
 (11, 3, False),
 (12, 2, False),
 (13, 2, False),
 (14, 2, False),
 (15, 2, False),
 (20, 1, False),
 (21, 1, False),
 (22, 1, False),
 (23, 1, False),
 (24, 1, False),
 (25, 1, False),
 (26, 1, False),
 (27, 1, False),
 (28, 1, False),
 (29, 1, False),
 (30, 1, False)]

Oh...

Up until this point I couldn't see how I'd constrain the max y value given it doesn't matter how high you chuck it - it's always going to come down. Thought I'd constrain it by valid days of x coordinates but turns it's possible to tweak x for it to arrive on any number of days because if you give it a an initial velocity of 6 or 7 it stops moving _in_ the x target area.

Was pretty sure this was a good approach.

It wasn't, back to the drawing board...

In [9]:
# Let's at least make a test to give us back the days an initial y velocity
# will land in the correct vertical zone
def test_y(target_area, y_vel):    
    day = 1
    valid = []
    while True:
        pos = y_pos(y_vel, day)

        if target_area.hit_y(pos):
            valid.append(day)

        overshoot = pos < target_area.min_y
        if overshoot:
            return valid

        day += 1

In [10]:
assert test_y(test_target_area, 2) == [7]
assert test_y(test_target_area, 3) == [9]
assert test_y(test_target_area, 0) == [4, 5]
assert test_y(test_target_area, -2) == [2, 3]
assert test_y(test_target_area, -11) == []

Got it! - what comes up, must come down - *to the original height*. If I know what day it comes back to the x-axis I can work out where it will be the next day and whether it will overshoot the target

In [11]:
def max_height(y_vel):
    '''Max height of the trajectory
    
    = y + (y - 1) + ... + (y - y)
    = y * (y + 1) / 2
    '''
    return int(y_vel * (y_vel + 1) / 2)

def days_to_x_axis(y_vel):
    '''Number of days till we hit the x-axis
    
    Goes up for y days, comes down for y days, hovers for 1
    '''
    return 2 * y_vel + 1

def next_pos_after_x_axis(y_vel):
    '''Y position the day after coming back to the horizontal'''
    num_days = days_to_x_axis(y_vel) + 1
    return y_pos(y_vel, num_days)

In [12]:
assert max_height(4) == 10
assert days_to_x_axis(2) == 5
assert next_pos_after_x_axis(3) == -4 

In [13]:
def y_range(target_area):
    '''A heuristic to give a range of possible valid initial y velocities hitting the target'''
    min_y = target_area.min_y
    max_y = 1

    while True:
        pos = next_pos_after_x_axis(max_y)
        max_y += 1
        if pos < target_area.min_y:
            # The rest will overshoot
            return range(min_y, max_y)


In [14]:
y_range(test_target_area)

range(-10, 11)

In [15]:
def part_1(target_area):
    for y in reversed(y_range(real_target_area)):
        if test_y(target_area, y):
            return max_height(y)


In [16]:
part_1(test_target_area), part_1(real_target_area)

(45, 13203)

### Day 17, part 2

How many distinct initial velocity values cause the probe to be within the target area after any step?

In [17]:
def y_velocities_by_day(target_area):
    y_positions = defaultdict(list)
    for y in y_range(target_area):
        valid = test_y(target_area, y)
        for day in valid:
            y_positions[day].append(y)
            
    return y_positions

In [18]:
# Tweak the above function to constrain the possible x velocities by the
# possible days we will be in the vertical zone (which is constrained)
def x_velocities_by_day(target_area, max_days):
    x_positions = defaultdict(list)
    min_vel = 1
    max_vel = target_area.max_x
    valid = []
    
    for vel in range(min_vel, max_vel + 1):
        day = 1
        while True:
            pos = x_pos(vel, day)
            if target_area.hit_x(pos):
                x_positions[day].append(vel)

            overshoot = pos > target_area.max_x
            
            if day > max_days or overshoot:
                break
            
            day += 1
    
    return x_positions

In [19]:
def distinct_positions(target_area):
    y_positions = y_velocities_by_day(target_area)
    max_days = max(y_positions.keys())
    x_positions = x_velocities_by_day(target_area, max_days)
    positions = []
    
    for day, y_values in y_positions.items():
        x_values = x_positions[day]
        for y in y_values:
            for x in x_values:
                positions.append((x, y))

    # Some initial positions can hit the target area multiple times
    return set(positions)

In [20]:
test_positions = distinct_positions(test_target_area)
len(test_positions)

112

In [21]:
test_positions = distinct_positions(test_target_area)
expected = {
(23,-10),  (25,-9),  (27,-5 ),  (29,-6 ),  (22,-6),  (21,-7 ),  (9,0   ),  (27,-7 ),  (24,-5),
(25,-7 ),  (26,-6),  (25,-5 ),  (6,8   ),  (11,-2),  (20,-5 ),  (29,-10),  (6,3   ),  (28,-7),
(8,0   ),  (30,-6),  (29,-8 ),  (20,-10),  (6,7  ),  (6,4   ),  (6,1   ),  (14,-4 ),  (21,-6),
(26,-10),  (7,-1 ),  (7,7   ),  (8,-1  ),  (21,-9),  (6,2   ),  (20,-7 ),  (30,-10),  (14,-3),
(20,-8 ),  (13,-2),  (7,3   ),  (28,-8 ),  (29,-9),  (15,-3 ),  (22,-5 ),  (26,-8 ),  (25,-8),
(25,-6 ),  (15,-4),  (9,-2  ),  (15,-2 ),  (12,-2),  (28,-9 ),  (12,-3 ),  (24,-6 ),  (23,-7),
(25,-10),  (7,8  ),  (11,-3 ),  (26,-7 ),  (7,1  ),  (23,-9 ),  (6,0   ),  (22,-10),  (27,-6),
(8,1   ),  (22,-8),  (13,-4 ),  (7,6   ),  (28,-6),  (11,-4 ),  (12,-4 ),  (26,-9 ),  (7,4),
(24,-10),  (23,-8),  (30,-8 ),  (7,0   ),  (9,-1 ),  (10,-1 ),  (26,-5 ),  (22,-9 ),  (6,5),
(7,5   ),  (23,-6),  (28,-10),  (10,-2 ),  (11,-1),  (20,-9 ),  (14,-2 ),  (29,-7 ),  (13,-3),
(23,-5 ),  (24,-8),  (27,-9 ),  (30,-7 ),  (28,-5),  (21,-10),  (7,9   ),  (6,6   ),  (21,-5),
(27,-10),  (7,2  ),  (30,-9 ),  (21,-8 ),  (22,-7),  (24,-9 ),  (20,-6 ),  (6,9   ),  (29,-5),
(8,-2  ),  (27,-8),  (30,-5 ),  (24,-7)
}
assert test_positions == expected

In [22]:
len(distinct_positions(real_target_area))

5644

### Day 17 Observations

I can't begin to say how much of a cock up I made of the second part. Hard coded target areas in functions, returning positions instead of velocities, not having re-run a cell (the big down side of notebooks!). Really should have been in a better position in part 2 after having done most of the work to establish x positioning when going down a dead end in part 1. It's funny how I always assume that everyone else sees the trick straight away and that no one else would ever have bugs in *their* code.

That being said - really nice puzzle. It's satisying that it's even possible to discover all the possible trajectories to hit an object with these physical constraints.

## Day 18 

https://adventofcode.com/2021/day/18

Add up all of the snailfish numbers from the homework assignment in the order they appear. What is the magnitude of the final sum?

In [3]:
test_input = get_puzzle_input('test_input_18.txt')
real_input = get_puzzle_input('input_18.txt')

In [4]:
class SnailFishNumber:
    def __init__(self, left, right):
        self.left = left
        self.right = right
        self.parent = None
        self.is_left_child = False
        self.depth = 1
    
    @staticmethod
    def from_string(string_repr):
        stack = []
        for char in string_repr:
            if char == '[':
                stack.append([])
            
            elif char == ']':
                curr_pair = stack.pop()
                num = SnailFishNumber(curr_pair[0], curr_pair[1])
                if not stack:
                    num._assign_parent()
                    return num
                stack[-1].append(num)
            
            elif char in '0123456789':
                stack[-1].append(int(char))
   
    @property
    def left_depth(self):
        if isinstance(self.left, int):
            return 0
        
        return 1 + self.left.depth

    @property
    def right_depth(self):
        if isinstance(self.right, int):
            return 1
        
        return 1 + self.right.depth
    
    @property
    def child_depth(self):
        return max(self.left_depth, self.right_depth)
    
    def _assign_parent(self):
        if not isinstance(self.left, int):
            self.left.parent = self
            self.left.is_left_child = True
            self.left.depth = self.depth + 1
            self.left._assign_parent()

        if not isinstance(self.right, int):
            self.right.parent = self
            self.right.is_left_child = False
            self.right.depth = self.depth + 1
            self.right._assign_parent()

    def replace_me(self, child):
        assert isinstance(child.left, int)
        assert isinstance(child.right, int)
        
        left_rel = self.first_left_relative_with_int()
        if left_rel:
            left_rel.left += child.left
            
        right_rel = self.first_right_relative_with_int()
        if right_rel:
            right_rel.right += child.right
        
        if child.is_left_child:
            self.left = 0
        else:
            self.right = 0

    def first_left_relative_with_int(self):
        fish = self
        
        while fish:
            if isinstance(fish.left, int):
                return fish
            
            fish = fish.parent

    def first_right_relative_with_int(self):
        fish = self
        
        while fish:
            if isinstance(fish.right, int):
                return fish
            
            fish = fish.parent

    def explode(self):
        left_exploded = False
        right_exploded = False

        if self.depth == 5:
            self.parent.replace_me(self)
            return True
        
        if not isinstance(self.left, int):
            left_exploded = self.left.explode()

        if not left_exploded and not isinstance(self.right, int):
            right_exploded = self.right.explode()
        
        return left_exploded or right_exploded

    
    def __repr__(self):
        return f'[{self.left},{self.right}]'
  

In [5]:
for expected in [
    '[1,2]',
    '[[1,2],3]',
    '[9,[8,7]]',
    '[[1,9],[8,5]]',
    '[[[[1,2],[3,4]],[[5,6],[7,8]]],9]',
    '[[[9,[3,8]],[[0,9],6]],[[[3,7],[4,9]],3]]',
    '[[[[1,3],[5,3]],[[1,3],[8,7]]],[[[4,9],[6,9]],[[8,2],[7,3]]]]',
]:
    assert str(SnailFishNumber.from_string(expected)) == expected

In [6]:
assert SnailFishNumber.from_string('[1,2]').child_depth == 1
assert SnailFishNumber.from_string('[[[[[9,8],1],2],3],4]').child_depth == 3

In [7]:
needs_exploding = SnailFishNumber.from_string('[[[[[9,8],1],2],3],4]')
node = needs_exploding.left.left.left.left
assert str(node) == '[9,8]'
assert node.depth == 5
assert node.is_left_child is True
assert node.parent.first_left_relative_with_int() is None
assert str(node.parent.first_right_relative_with_int()) == '[[9,8],1]'

In [8]:
num = SnailFishNumber.from_string('[[[[[9,8],1],2],3],4]')
num.explode()
assert str(num) == '[[[[0,9],2],3],4]'

num = SnailFishNumber.from_string('[7,[6,[5,[4,[3,2]]]]]')
num.explode()
assert str(num) == '[7,[6,[5,[7,0]]]]'

num = SnailFishNumber.from_string('[[6,[5,[4,[3,2]]]],1]')
num.explode()
assert str(num) == '[[6,[5,[7,0]]],3]'

num = SnailFishNumber.from_string('[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]')
num.explode()
assert str(num) == '[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]'


AssertionError: 

Uh oh. in `[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]` I have to add the 3 to the 6 and can't tell how I work out that that is the next to the right in the nested structure. Scrap that approach

In [15]:
@dataclass(frozen=True)
class NestedList:
    nest: list

    def map(self, func):
        def _apply(item, func):
            if isinstance(item, list):
                return [_apply(x, func) for x in item]
            else:
                return func(item)
        
        return NestedList(_apply(self.nest, func))
    
    def reduce(self, func):
        def _reduce(item, func):
            if isinstance(item, list):
                return func([
                    _reduce(x, func)
                    for x in item
                ])
            else:
                return item
            
        return _reduce(self.nest, func)

assert NestedList([1, 2, [3, 4]]).map(lambda x: x + 1) == NestedList([2, 3, [4, 5]])
assert NestedList([1, 2, [3, 4]]).reduce(sum) == 10
assert NestedList([[5, 6], [3, 4]]).reduce(lambda x: 3*x[0] + 2*x[1]) == 115

In [14]:
NestedList([[5, 6], [3, 4]]).reduce(lambda x: 3*x[0] + 2*x[1])

115

In [39]:
@dataclass(frozen=True)
class FishNum:
    string_repr: str

    def __post_init__(self):
        self._explode()
        
    def to_simplest_form(self):
        fish_num = self
        while True:
            new = fish_num.explode()
            if new == fish_num:
                return fish_num
            #print(new)
            fish_num = new
     
    def _explode(self):
        output = []
        nested_structure, refs, exploded = self.explode_as_refs()
        
        nested_list = NestedList(nested_structure)
        lookup_func = lambda x: refs[x] if x is not None else 0
        nested_form = nested_list.map(lookup_func)
        
        # Hack after re-reading the question
        split_found = False
        def split_func(x):
            nonlocal split_found
            if x < 10 or split_found:
                return x
            else:
                split_found = True
                return [math.floor(x/2), math.ceil(x/2)]
            
        
        if not exploded:
            #split_func = lambda x: x if x < 10 else [math.floor(x/2), math.ceil(x/2)]
            nested_form = nested_form.map(split_func)

        object.__setattr__(self, 'nested_form', nested_form)
        
    def explode(self):
        str_repr = str(self.nested_form.nest).replace(" ", "")
        return FishNum(str_repr)
        
    def explode_as_refs(self):
        stack = []
        num_list = []
        stack_depth = 0
        exploded = False
        right_exploded = 0
        current_num = ''
        for char in self.string_repr:
            if char == '[':
                stack_depth += 1
                stack.append([])

            elif char == ',':
                num = int(current_num) if current_num else None
                current_num = ''

                if num is not None:
                    if right_exploded:
                        num += right_exploded
                        right_exploded = 0
                    # Pass by reference
                    stack[-1].append(len(num_list))
                    num_list.append(num)

            elif char == ']':
                num = int(current_num) if current_num else None
                current_num = ''
                exploding = not exploded and stack_depth == 5

                if num is not None:
                    if right_exploded:
                        num += right_exploded
                        right_exploded = 0
                    stack[-1].append(len(num_list))
                    num_list.append(num)

                pair = stack.pop()

                if not stack:
                    return pair, num_list, exploded

                stack_depth -= 1

                if not exploding:
                    stack[-1].append(pair)
                    continue

                # exploding
                right = num_list.pop()
                left = num_list.pop()
                stack[-1].append(None)

                if num_list:
                    num_list[-1] += left
                right_exploded = right
                exploded = True

                stack_depth -= 1

            elif char in '0123456789':
                current_num += char

    @property
    def magnitude(self):
        return self.nested_form.reduce(lambda x: 3*x[0] + 2*x[-1])
        

    def __add__(self, other):
        return FishNum('[' + self.string_repr + ',' + other.string_repr + ']')


In [40]:
assert FishNum('[[1,2],3]').explode() == FishNum('[[1,2],3]')
assert FishNum('[[[[[9,8],1],2],3],4]').explode() == FishNum('[[[[0,9],2],3],4]')
assert FishNum('[7,[6,[5,[4,[3,2]]]]]').explode() == FishNum('[7,[6,[5,[7,0]]]]')
assert FishNum('[[6,[5,[4,[3,2]]]],1]').explode() == FishNum('[[6,[5,[7,0]]],3]')
num = FishNum('[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]')
assert num.explode() == FishNum('[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]')
assert num.explode().explode() == FishNum('[[3,[2,[8,0]]],[9,[5,[7,0]]]]')

In [41]:
assert FishNum('[[9,1],[1,9]]').magnitude == 129
assert FishNum('[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]').magnitude == 3488

In [42]:
def part_1(lines):
    result = None
    for line in lines:
        number = FishNum(line)
        if result:
            result += number
        else:
            result = number
        result = result.to_simplest_form()
    
    return result 

In [43]:
test_1 =['[1,1]', '[2,2]', '[3,3]', '[4,4]']
assert part_1(test_1) == FishNum('[[[[1,1],[2,2]],[3,3]],[4,4]]')

In [44]:
test_2 =['[1,1]', '[2,2]', '[3,3]', '[4,4]', '[5,5]']
assert part_1(test_2) == FishNum('[[[[3,0],[5,3]],[4,4]],[5,5]]')

In [45]:
test_3 = ['[1,1]', '[2,2]', '[3,3]', '[4,4]', '[5,5]', '[6,6]']
assert part_1(test_3) == FishNum('[[[[5,0],[7,4]],[5,5]],[6,6]]')

In [46]:
test_4 = [
    '[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]',
    '[[[5,[2,8]],4],[5,[[9,9],0]]]',
    '[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]',
    '[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]',
    '[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]',
    '[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]',
    '[[[[5,4],[7,7]],8],[[8,3],8]]',
    '[[9,3],[[9,9],[6,[4,9]]]]',
    '[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]',
    '[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]',
]

In [47]:
part_1(test_4).magnitude

4140

In [48]:
part_1(real_input).magnitude

4088

### Day 18 Part 2 

What is the largest magnitude of any sum of two different snailfish numbers from the homework assignment?

In [49]:
from itertools import permutations

In [50]:
def part_2(input):
    numbers = [FishNum(x) for x in input]
    perms = permutations(numbers, 2)
    max = 0
    
    for a, b in perms:
        mag = (a + b).to_simplest_form().magnitude
        if mag > max:
            max = mag
    
    return max

In [51]:
part_2(test_4)

3993

In [52]:
part_2(real_input)

4536

### Day 18 Observations

That was painful. FOR GOD SAKE - READ THE QUESTION. Two huge things I missed:
- You only apply the split if there are no more explosions
- You don't apply all the splits at once.

It's one of those puzzles that looks relatively easy. Who knows, perhaps it is. Wasn't for me!

Oh man, never again.